In [ ]:
#data
import pandas as pd
import numpy as np

#text
from bs4 import BeautifulSoup
import re

#requests
import requests

#aux
import PyPDF2
import os

### extrair dados de questão de prova

- os seguintes dados serão os atributos do csv com as questões de prova
    - ano;
    - cor da prova;
    - digital ou papel;
    - área da questão;
    - numero da questão;
    - texto completo;
    - texto base com comandos e referências (quando possível);
    - texto da alternativa A da questão (quando possível);
    - texto da alternativa B da questão (quando possível);
    - texto da alternativa C da questão (quando possível);
    - texto da alternativa D da questão (quando possível);
    - texto da alternativa E da questão (quando possível);

### ENEM 2009

In [ ]:
#aplicado a 2009
def extrai_questoes_1(dir_, ano, pdf, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    #txt_pdf = txt_pdf.replace('\n', '')
    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')

    txt_pdf  = re.sub(r"CH – 1º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"CN – 1º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"LC – 1º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"MT – 1º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"CH – 2º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"CN – 2º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"LC – 2º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"MT – 2º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"RED – 2º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"RED – 1º dia(.*?)ENEM 2009", '', txt_pdf, re.DOTALL)
    txt_pdf  = re.sub(r"Transcreva(.*?)Folha de Redação", '', txt_pdf,  re.DOTALL)
    txt_pdf  = txt_pdf.replace("1  \n2  \n3  \n4  \n5  \n6  \n7  \n8  \n9  \n10  \n11  \n12  \n13  \n14  \n15  \n16  \n17  \n18  \n19  \n20  \n21  \n22  \n23  \n24  \n25  \n26  \n27  \n28  \n29  \n30  \n", '')
    txt_pdf  = txt_pdf.replace("Transcreva a sua Redação para a Folha de Redação", '')
    txt_pdf = txt_pdf.replace('Rascunho', '')
    letras = ['\nA ', '\nB ',  '\nC ','\nD ', '\nE ']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            pattern = re.compile(r"Questão "+str(i)+"(.*?)Questão "+str(i+1), re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0].strip()
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = texto_base[0].replace('\n',' ')
                texto_base = re.sub(r'\s+', ' ', texto_base, flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print(questao)
        else:
            pattern = re.compile(r"Questão "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = texto_base[0].replace('\n',' ')
                texto_base = re.sub(r'\s+', ' ', texto_base, flags=re.DOTALL)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', ' ')
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_A', 8: 'alternativa_B', 9: 'alternativa_C', 10: 'alternativa_D', 11: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
file_path, df_dados_ENEM_2009_PROVA_DIA_1_AMARELO_2 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_1_AMARELO_2.pdf', 'AMARELO', 'PAPEL', 'CN', 1, 90)

In [ ]:
df_dados_ENEM_2009_PROVA_DIA_1_AMARELO_2.head(2)

In [ ]:
df_dados_ENEM_2009_PROVA_DIA_1_AMARELO_2.loc[df_dados_ENEM_2009_PROVA_DIA_1_AMARELO_2['num_questao'] > 45, 'area'] = 'CH'
print(df_dados_ENEM_2009_PROVA_DIA_1_AMARELO_2.area.unique())
df_dados_ENEM_2009_PROVA_DIA_1_AMARELO_2.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2009_PROVA_DIA_1_AZUL_1 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_1_AZUL_1.pdf', 'AZUL', 'PAPEL', 'CN', 1, 90)

In [ ]:
df_ENEM_2009_PROVA_DIA_1_AZUL_1.head(2)

In [ ]:
df_ENEM_2009_PROVA_DIA_1_AZUL_1.loc[df_ENEM_2009_PROVA_DIA_1_AZUL_1['num_questao'] > 45, 'area'] = 'CH'
print(df_ENEM_2009_PROVA_DIA_1_AZUL_1.area.unique())
df_ENEM_2009_PROVA_DIA_1_AZUL_1.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2009_PROVA_DIA_1_BRANCO_3 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_1_BRANCO_3.pdf', 'BRANCO', 'PAPEL', 'CN', 1, 90)

In [ ]:
df_ENEM_2009_PROVA_DIA_1_BRANCO_3.head(2)

In [ ]:
df_ENEM_2009_PROVA_DIA_1_BRANCO_3.loc[df_ENEM_2009_PROVA_DIA_1_BRANCO_3['num_questao'] > 45, 'area'] = 'CH'
print(df_ENEM_2009_PROVA_DIA_1_BRANCO_3.area.unique())
df_ENEM_2009_PROVA_DIA_1_BRANCO_3.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2009_PROVA_DIA_1_ROSA_4 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_1_ROSA_4.pdf', 'ROSA', 'PAPEL', 'CN', 1, 90)

In [ ]:
df_ENEM_2009_PROVA_DIA_1_ROSA_4.head(2)

In [ ]:
df_ENEM_2009_PROVA_DIA_1_ROSA_4.loc[df_ENEM_2009_PROVA_DIA_1_ROSA_4['num_questao'] > 45, 'area'] = 'CH'
print(df_ENEM_2009_PROVA_DIA_1_ROSA_4.area.unique())
df_ENEM_2009_PROVA_DIA_1_ROSA_4.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2009_PROVA_DIA_2_AMARELO_5 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_2_AMARELO_5.pdf', 'AMARELO', 'PAPEL', 'LC', 91, 180)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_AMARELO_5.head(2)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_AMARELO_5.loc[df_ENEM_2009_PROVA_DIA_2_AMARELO_5['num_questao'] > 136, 'area'] = 'MT'
print(df_ENEM_2009_PROVA_DIA_2_AMARELO_5.area.unique())
df_ENEM_2009_PROVA_DIA_2_AMARELO_5.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2009_PROVA_DIA_2_AZUL_7 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_2_AZUL_7.pdf', 'AZUL', 'PAPEL', 'LC', 91, 180)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_AZUL_7.head(2)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_AZUL_7.loc[df_ENEM_2009_PROVA_DIA_2_AZUL_7['num_questao'] > 136, 'area'] = 'MT'
print(df_ENEM_2009_PROVA_DIA_2_AZUL_7.area.unique())
df_ENEM_2009_PROVA_DIA_2_AZUL_7.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2009_PROVA_DIA_2_CINZA_6 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_2_CINZA_6.pdf', 'CINZA', 'PAPEL', 'LC', 91, 180)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_CINZA_6.head(2)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_CINZA_6.loc[df_ENEM_2009_PROVA_DIA_2_CINZA_6['num_questao'] > 136, 'area'] = 'MT'
print(df_ENEM_2009_PROVA_DIA_2_CINZA_6.area.unique())
df_ENEM_2009_PROVA_DIA_2_CINZA_6.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2009_PROVA_DIA_2_ROSA_8 = extrai_questoes_1('_itens', 2009, 'ENEM_2009_PROVA_DIA_2_ROSA_8.pdf', 'ROSA', 'PAPEL', 'LC', 91, 180)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_ROSA_8.head(2)

In [ ]:
df_ENEM_2009_PROVA_DIA_2_ROSA_8.loc[df_ENEM_2009_PROVA_DIA_2_ROSA_8['num_questao'] > 136, 'area'] = 'MT'
print(df_ENEM_2009_PROVA_DIA_2_ROSA_8.area.unique())
df_ENEM_2009_PROVA_DIA_2_ROSA_8.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2009 = pd.concat([df_dados_ENEM_2009_PROVA_DIA_1_AMARELO_2, 
                           df_ENEM_2009_PROVA_DIA_1_AZUL_1, 
                           df_ENEM_2009_PROVA_DIA_1_BRANCO_3, 
                           df_ENEM_2009_PROVA_DIA_1_ROSA_4, 
                           df_ENEM_2009_PROVA_DIA_2_AMARELO_5,
                           df_ENEM_2009_PROVA_DIA_2_AZUL_7,
                           df_ENEM_2009_PROVA_DIA_2_CINZA_6,
                           df_ENEM_2009_PROVA_DIA_2_ROSA_8], ignore_index=True, axis = 0)

# Resultado
df_prova_2009.shape

In [ ]:
#geração da base de textos de 2009
file = os.path.join('_itens', str(2009), '2009_ENEM_itens')
df_prova_2009.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_prova_2009.head(60)

In [ ]:
file = os.path.join('_itens', str(2009), 'ITENS_PROVA_2009')
df_itens_2009 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2009.head()

In [ ]:
df_itens_2009.loc[df_itens_2009['SG_AREA'] == 'CH', 'SG_AREA'] = 'CN_'
df_itens_2009.loc[df_itens_2009['SG_AREA'] == 'CN', 'SG_AREA'] = 'CH' 
df_itens_2009.loc[df_itens_2009['SG_AREA'] == 'CN_', 'SG_AREA'] = 'CN'

In [ ]:
df_itens_2009['id'] = df_itens_2009['CO_POSICAO'].apply(str) + '_' + df_itens_2009['SG_AREA'].apply(str) + '_' + df_itens_2009['TX_COR'].apply(str)

In [ ]:
df_prova_2009['id'] = df_prova_2009['num_questao'].apply(str) + '_' + df_prova_2009['area'].apply(str) + '_' + df_prova_2009['cor'].apply(str)

In [ ]:
df_2009_merge = pd.merge(df_itens_2009, df_prova_2009, on='id', how='left')

In [ ]:
df_2009_merge.head(5)

In [ ]:
for col in df_2009_merge.columns:
    df_2009_merge = df_2009_merge.rename(columns = {col:col.lower()})

In [ ]:
file = os.path.join('_itens', str(2009), '2009_ENEM_COMPLETA')
df_2009_merge.to_csv(file + '.csv', sep = ';', index = False)

### ENEM 2010

In [ ]:
#aplicado a 2010
def extrai_questoes_2(dir_, ano, pdf, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf  = re.sub(r'g\d+', '', txt_pdf)
    txt_pdf = re.sub('  ', '', txt_pdf)
    txt_pdf = re.sub(" +", " ", txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = txt_pdf.replace('   ', '')
    txt_pdf = txt_pdf.replace('      ', '')
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    txt_pdf = re.sub(r'\*azul75sab\d+\*', '', txt_pdf)
    txt_pdf = re.sub(r'\*azul25dom\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('LC - 2º dia|Caderno 7 - AZUL- Página 31', '')
    txt_pdf  = re.sub("2010\nCH - 1º dia\|Caderno 1 - AZUL - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"2010\nCN - 1º dia\|Caderno 1 - AZUL - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub("2010\nLC - 2º dia\|Caderno 7 - AZUL- Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"2010\nMT - 2º dia\|Caderno 7 - AZUL- Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('Rascunho', '')
    letras = ['\nA ', '\nB ',  '\nC ','\nD ', '\nE ']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            pattern = re.compile(r"Questão "+str(i)+"(.*?)Questão "+str(i+1), re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0].strip()
                questao = re.sub(r'\*azul75sab\d+\*', '', questao)
                
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = texto_base[0].replace('\n',' ')
                texto_base = re.sub(r'\s+', ' ', texto_base, flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print(questao)
        else:
            pattern = re.compile(r"Questão "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = texto_base[0].replace('\n',' ').strip()
                texto_base = re.sub(r'\s+', ' ', texto_base, flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', ' ')
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_A', 8: 'alternativa_B', 9: 'alternativa_C', 10: 'alternativa_D', 11: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
#aplicado a 2010
def extrai_questoes_html_1(dir_, ano, arq, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    file_path = os.path.join(dir_, str(ano), arq)
    
    for i in range(num_primeira_questao, num_ultima_questao):
        print('questao', i)
        url = "http://educacao.globo.com/provas/enem-2010/questoes/"+str(i)+".html"
        # faz a requisição para a url
        response = requests.get(url)
        # verifica se a requisição foi bem sucedida
        if response.status_code == 200:
            #print(response.text)
            # transforma a página em um objeto BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")
            # procura pelo conteúdo
            pattern = re.compile(r"<span class=\"numero\">Questão "+str(i)+"</span>(.*?)separador separador-resolucao", re.DOTALL)
            questao = pattern.findall(response.text)

            if (len(questao) > 0):
                pattern = re.compile(r"<div class=\"texto\">(.*?)</div>", re.DOTALL)
                texto_base = pattern.findall(questao[0])

                pattern = re.compile(r"<div class=\"enunciado\">(.*?)</div>", re.DOTALL)
                enunciado = pattern.findall(questao[0])

                pattern = re.compile(r"<div class=\"letra\">[A-E]+</div>(.*?)</div>", re.DOTALL)
                alternativas = pattern.findall(questao[0])
                alternativa_a = alternativas[0]
                alternativa_b = alternativas[1]
                alternativa_c = alternativas[2]
                alternativa_d = alternativas[3]
                alternativa_e = alternativas[4]

                questao = re.sub(re.compile('<.*?>'), '', questao[0]).replace('\n', '').strip().replace('<hr class="', '')
                questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
                texto_base = re.sub(re.compile('<.*?>'), '', texto_base[0]).replace('\n', '').strip()
                enunciado = re.sub(re.compile('<.*?>'), '',enunciado[0]).replace('\n', '').strip()
                texto_base = texto_base + ' ' + enunciado
                text_base = re.sub(r'\s+', ' ', texto_base, flags=re.DOTALL)
                alternativa_a = re.sub(re.compile('<.*?>'), '', alternativa_a).replace('\n', '')
                alternativa_a = re.sub(r'\s+', ' ', alternativa_a, flags=re.DOTALL)
                alternativa_b = re.sub(re.compile('<.*?>'), '', alternativa_b).replace('\n', '')
                alternativa_b = re.sub(r'\s+', ' ', alternativa_b, flags=re.DOTALL)
                alternativa_c = re.sub(re.compile('<.*?>'), '', alternativa_c).replace('\n', '')
                alternativa_c = re.sub(r'\s+', ' ', alternativa_c, flags=re.DOTALL)
                alternativa_d = re.sub(re.compile('<.*?>'), '', alternativa_d).replace('\n', '')
                alternativa_d = re.sub(r'\s+', ' ', alternativa_d, flags=re.DOTALL)
                alternativa_e = re.sub(re.compile('<.*?>'), '', alternativa_e).replace('\n', '')
                alternativa_e = re.sub(r'\s+', ' ', alternativa_e, flags=re.DOTALL)

                # escreve o conteúdo em um arquivo txt
                with open(file_path+".txt", "w",  encoding="utf-8") as f:

                    f.write(questao)

                dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, alternativa_a, alternativa_b, alternativa_c, alternativa_d, alternativa_e])    
                print([ano, cor, meio_aplicacao, i, area, questao, texto_base, alternativa_a, alternativa_b, alternativa_c, alternativa_d, alternativa_e]) 
            else:
                print('questao vazia')
        # se a requisição não foi bem sucedida, imprime uma mensagem de erro
        else:
            print("Erro ao obter a página:", response.status_code) 
            
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_a', 8: 'alternativa_b', 9: 'alternativa_c', 10: 'alternativa_d', 11: 'alternativa_e'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
file_path, df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1 = extrai_questoes_html_1('_itens', 2010, 'ENEM_2010_PROVA_GAB_DIA_1_AZUL_1(2_Aplicação).pdf', 'AZUL', 'PAPEL', 'CN', 1, 90)

In [ ]:
df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1.head(2)

In [ ]:
df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1.loc[df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1['num_questao'] <= 45, 'area'] = 'CH'
df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1.loc[df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1['num_questao'] > 45, 'area'] = 'CN'
print(df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1.area.unique())

In [ ]:
file_path, df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1 = extrai_questoes_html_1('_itens', 2010, 'ENEM_2010_PROVA_GAB_DIA_2_AZUL_1(2_Aplicação).pdf', 'AZUL', 'PAPEL', 'CN', 90, 181)

In [ ]:
df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1.head(2)

In [ ]:
df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1.loc[df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1['num_questao'] <= 135, 'area'] = 'LC'
df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1.loc[df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1['num_questao'] > 135, 'area'] = 'MT'
print(df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1.area.unique())
df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2010 = pd.concat([df_ENEM_2010_PROVA_GAB_DIA_1_AZUL_1, df_ENEM_2010_PROVA_GAB_DIA_2_AZUL_1], ignore_index=True, axis = 0)

# Resultado
df_prova_2010.shape

In [ ]:
#geração da base de textos de 2009
file = os.path.join('_itens', str(2010), '2010_ENEM_itens')
df_prova_2010.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_prova_2010.head()

In [ ]:
file = os.path.join('_itens', str(2010), 'ITENS_PROVA_2010')
df_itens_2010 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2010.head()

In [ ]:
df_itens_2010['TX_COR'] = df_itens_2010['TX_COR'].str.upper()

In [ ]:
df_itens_2010['id'] = df_itens_2010['CO_POSICAO'].apply(str) + '_' + df_itens_2010['SG_AREA'].apply(str) + '_' + df_itens_2010['TX_COR'].apply(str)

In [ ]:
df_prova_2010['id'] = df_prova_2010['num_questao'].apply(str) + '_' + df_prova_2010['area'].apply(str) + '_' + df_prova_2010['cor'].apply(str)

In [ ]:
df_2010_merge = pd.merge(df_itens_2010, df_prova_2010, on='id', how='left')

In [ ]:
for col in df_2010_merge.columns:
    df_2010_merge = df_2010_merge.rename(columns = {col:col.lower()})

In [ ]:
file = os.path.join('_itens', str(2010), '2010_ENEM_COMPLETA')
df_2010_merge.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_2010_merge.head()

### ENEM 2011

In [ ]:
#aplicado a 2011
def extrai_questoes_html_2(dir_, ano, arq, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    file_path = os.path.join(dir_, str(ano), arq)
    
    for i in range(num_primeira_questao, num_ultima_questao):
        print('questao', i)
        url = "http://educacao.globo.com/provas/enem-2011/questoes/"+(str(0)+str(i) if i < 10 else str(i))+".html"
        print(url)
        # faz a requisição para a url
        response = requests.get(url)
        # verifica se a requisição foi bem sucedida
        if response.status_code == 200:
            #print(response.text)
            # transforma a página em um objeto BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")
            # procura pelo conteúdo
            pattern = re.compile(r"<span class=\"numero\">Questão "+(str(0)+str(i) if i < 10 else str(i))+"</span>(.*?)separador separador-resolucao", re.DOTALL)
            questao = pattern.findall(response.text)

            if (len(questao) > 0):
                pattern = re.compile(r"<div class=\"texto\">(.*?)</div>", re.DOTALL)
                texto_base = pattern.findall(questao[0])

                pattern = re.compile(r"<div class=\"enunciado\">(.*?)</div>", re.DOTALL)
                enunciado = pattern.findall(questao[0])

                pattern = re.compile(r"<div class=\"letra\">[A-E]+</div>(.*?)</div>", re.DOTALL)
                alternativas = pattern.findall(questao[0])
                alternativa_a = alternativas[0]
                alternativa_b = alternativas[1]
                alternativa_c = alternativas[2]
                alternativa_d = alternativas[3]
                alternativa_e = alternativas[4]

                questao = re.sub(re.compile('<.*?>'), '', questao[0]).replace('\n', '').strip().replace('<hr class="', '')
                questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
                texto_base = re.sub(re.compile('<.*?>'), '', texto_base[0]).replace('\n', '').strip()
                enunciado = re.sub(re.compile('<.*?>'), '',enunciado[0]).replace('\n', '').strip()
                texto_base = texto_base + ' ' + enunciado
                text_base = re.sub(r'\s+', ' ', texto_base, flags=re.DOTALL)
                alternativa_a = re.sub(re.compile('<.*?>'), '', alternativa_a).replace('\n', '')
                alternativa_a = re.sub(r'\s+', ' ', alternativa_a, flags=re.DOTALL)
                alternativa_b = re.sub(re.compile('<.*?>'), '', alternativa_b).replace('\n', '')
                alternativa_b = re.sub(r'\s+', ' ', alternativa_b, flags=re.DOTALL)
                alternativa_c = re.sub(re.compile('<.*?>'), '', alternativa_c).replace('\n', '')
                alternativa_c = re.sub(r'\s+', ' ', alternativa_c, flags=re.DOTALL)
                alternativa_d = re.sub(re.compile('<.*?>'), '', alternativa_d).replace('\n', '')
                alternativa_d = re.sub(r'\s+', ' ', alternativa_d, flags=re.DOTALL)
                alternativa_e = re.sub(re.compile('<.*?>'), '', alternativa_e).replace('\n', '')
                alternativa_e = re.sub(r'\s+', ' ', alternativa_e, flags=re.DOTALL)

                # escreve o conteúdo em um arquivo txt
                with open(file_path+".txt", "w",  encoding="utf-8") as f:

                    f.write(questao)

                dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, alternativa_a, alternativa_b, alternativa_c, alternativa_d, alternativa_e])    
                print([ano, cor, meio_aplicacao, i, area, questao, texto_base, alternativa_a, alternativa_b, alternativa_c, alternativa_d, alternativa_e]) 
            else:
                print('questao vazia')
        # se a requisição não foi bem sucedida, imprime uma mensagem de erro
        else:
            print("Erro ao obter a página:", response.status_code) 
            
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_a', 8: 'alternativa_b', 9: 'alternativa_c', 10: 'alternativa_d', 11: 'alternativa_e'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
file_path, df_ENEM_2011_CAD_AZUL = extrai_questoes_html_2('_itens', 2011, 'ENEM_2011_CAD_AZUL', 'AZUL', 'PAPEL', 'CH', 1, 181)

In [ ]:
df_ENEM_2011_CAD_AZUL.shape

In [ ]:
df_ENEM_2011_CAD_AZUL.loc[df_ENEM_2011_CAD_AZUL['num_questao'] <= 45, 'area'] = 'CH'
df_ENEM_2011_CAD_AZUL.loc[df_ENEM_2011_CAD_AZUL['num_questao'] > 45, 'area'] = 'CN'
print(df_ENEM_2011_CAD_AZUL.area.unique())
df_ENEM_2011_CAD_AZUL.loc[(df_ENEM_2011_CAD_AZUL['num_questao'] <= 135) & (df_ENEM_2011_CAD_AZUL['num_questao'] >= 91), 'area'] = 'LC'
df_ENEM_2011_CAD_AZUL.loc[df_ENEM_2011_CAD_AZUL['num_questao'] > 135, 'area'] = 'MT'
print(df_ENEM_2011_CAD_AZUL.area.unique())
df_ENEM_2011_CAD_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
df_ENEM_2011_CAD_AZUL.head()

In [ ]:
df_ENEM_2011_CAD_AZUL['cor'] = 'Azul'

In [ ]:
file = os.path.join('_itens', str(2011), 'ITENS_PROVA_2011')
df_itens_2011 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2011.head()

In [ ]:
df_itens_2011['id'] = df_itens_2011['CO_POSICAO'].apply(str) + '_' + df_itens_2011['SG_AREA'].apply(str) + '_' + df_itens_2011['TX_COR'].apply(str)

In [ ]:
df_ENEM_2011_CAD_AZUL['id'] = df_ENEM_2011_CAD_AZUL['num_questao'].apply(str) + '_' + df_ENEM_2011_CAD_AZUL['area'].apply(str) + '_' + df_ENEM_2011_CAD_AZUL['cor'].apply(str)

In [ ]:
df_ENEM_2011_CAD_AZUL.id.unique()

In [ ]:
df_2011_merge = pd.merge(df_itens_2011, df_ENEM_2011_CAD_AZUL, on='id', how='left')

In [ ]:
for col in df_2011_merge.columns:
    df_2011_merge = df_2011_merge.rename(columns = {col:col.lower()})

In [ ]:
file = os.path.join('_itens', str(2011), '2011_ENEM_COMPLETA')
df_2011_merge.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_2011_merge.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_2011_merge.head()

### 2012

In [ ]:
#aplicado a 2012
def extrai_questoes_3(dir_, ano, pdf, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf  = re.sub(r'g\d+', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    txt_pdf = re.sub(r'\*((AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))((25)|(75))((dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('23456789101112131415161718192021222324252627282930', '')
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    letras = ['A\n', 'B\n',  'C\n ','D\n ', 'E\n']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print(questao)
        else:
            pattern = re.compile(r"QUESTÃO "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_A', 8: 'alternativa_B', 9: 'alternativa_C', 10: 'alternativa_D', 11: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
file_path, df_caderno_enem2012_dom_amarelo = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_dom_amarelo.pdf', 'Amarelo', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_caderno_enem2012_dom_amarelo.head(2)

In [ ]:
df_caderno_enem2012_dom_amarelo.loc[df_caderno_enem2012_dom_amarelo['num_questao'] <= 135, 'area'] = 'LC'
df_caderno_enem2012_dom_amarelo.loc[df_caderno_enem2012_dom_amarelo['num_questao'] > 135, 'area'] = 'MT'
print(df_caderno_enem2012_dom_amarelo.area.unique())
df_caderno_enem2012_dom_amarelo.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_caderno_enem2012_dom_azul = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_dom_azul.pdf', 'Azul', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_caderno_enem2012_dom_azul.tail(20)

In [ ]:
df_caderno_enem2012_dom_azul.loc[df_caderno_enem2012_dom_azul['num_questao'] <= 135, 'area'] = 'LC'
df_caderno_enem2012_dom_azul.loc[df_caderno_enem2012_dom_azul['num_questao'] > 135, 'area'] = 'MT'
print(df_caderno_enem2012_dom_azul.area.unique())
df_caderno_enem2012_dom_azul.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_caderno_enem2012_dom_cinza = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_dom_cinza.pdf', 'Cinza', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_caderno_enem2012_dom_cinza.tail(20)

In [ ]:
df_caderno_enem2012_dom_cinza.loc[df_caderno_enem2012_dom_cinza['num_questao'] <= 135, 'area'] = 'LC'
df_caderno_enem2012_dom_cinza.loc[df_caderno_enem2012_dom_cinza['num_questao'] > 135, 'area'] = 'MT'
print(df_caderno_enem2012_dom_cinza.area.unique())
df_caderno_enem2012_dom_cinza.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:

file_path, df_caderno_enem2012_dom_rosa = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_dom_rosa.pdf', 'Rosa', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_caderno_enem2012_dom_rosa.tail(20)

In [ ]:
df_caderno_enem2012_dom_rosa.loc[df_caderno_enem2012_dom_rosa['num_questao'] <= 135, 'area'] = 'LC'
df_caderno_enem2012_dom_rosa.loc[df_caderno_enem2012_dom_rosa['num_questao'] > 135, 'area'] = 'MT'
print(df_caderno_enem2012_dom_rosa.area.unique())
df_caderno_enem2012_dom_rosa.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_caderno_enem2012_sab_amarelo = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_sab_amarelo.pdf', 'Amarelo', 'PAPEL', 'LC', 1, 90)

In [ ]:
df_caderno_enem2012_sab_amarelo.loc[df_caderno_enem2012_sab_amarelo['num_questao'] <= 45, 'area'] = 'CH'
df_caderno_enem2012_sab_amarelo.loc[df_caderno_enem2012_sab_amarelo['num_questao'] > 45, 'area'] = 'CN'
print(df_caderno_enem2012_sab_amarelo.area.unique())
df_caderno_enem2012_sab_amarelo.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_caderno_enem2012_sab_azul = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_sab_azul.pdf', 'Azul', 'PAPEL', 'CH', 1, 90)

In [ ]:
df_caderno_enem2012_sab_azul.loc[df_caderno_enem2012_sab_azul['num_questao'] <= 45, 'area'] = 'CH'
df_caderno_enem2012_sab_azul.loc[df_caderno_enem2012_sab_azul['num_questao'] > 45, 'area'] = 'CN'
print(df_caderno_enem2012_sab_azul.area.unique())
df_caderno_enem2012_sab_azul.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_caderno_enem2012_sab_branco = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_sab_branco.pdf', 'Branco', 'PAPEL', 'CH', 1, 90)

In [ ]:
df_caderno_enem2012_sab_branco.loc[df_caderno_enem2012_sab_branco['num_questao'] <= 45, 'area'] = 'CH'
df_caderno_enem2012_sab_branco.loc[df_caderno_enem2012_sab_branco['num_questao'] > 45, 'area'] = 'CN'
print(df_caderno_enem2012_sab_branco.area.unique())
df_caderno_enem2012_sab_branco.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_caderno_enem2012_sab_rosa = extrai_questoes_3('_itens', 2012, 'caderno_enem2012_sab_rosa.pdf', 'Rosa', 'PAPEL', 'CN', 1, 90)

In [ ]:
df_caderno_enem2012_sab_rosa.loc[df_caderno_enem2012_sab_rosa['num_questao'] <= 45, 'area'] = 'CH'
df_caderno_enem2012_sab_rosa.loc[df_caderno_enem2012_sab_rosa['num_questao'] > 45, 'area'] = 'CN'
print(df_caderno_enem2012_sab_rosa.area.unique())
df_caderno_enem2012_sab_rosa.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2012 = pd.concat([df_caderno_enem2012_dom_amarelo, 
                           df_caderno_enem2012_dom_azul, 
                           df_caderno_enem2012_dom_cinza,
                           df_caderno_enem2012_dom_rosa,
                           df_caderno_enem2012_sab_amarelo,
                           df_caderno_enem2012_sab_azul,
                           df_caderno_enem2012_sab_branco,
                           df_caderno_enem2012_sab_rosa], ignore_index=True, axis = 0)

# Resultado
df_prova_2012.shape

In [ ]:
df_prova_2012.tail()

In [ ]:
file = os.path.join('_itens', str(2012), 'ITENS_PROVA_2012')
df_itens_2012 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')

In [ ]:
df_itens_2012.head()

In [ ]:
df_itens_2012['id'] = df_itens_2012['CO_POSICAO'].apply(str) + '_' + df_itens_2012['SG_AREA'].apply(str) + '_' + df_itens_2012['TX_COR'].apply(str)
df_prova_2012['id'] = df_prova_2012['num_questao'].apply(str) + '_' + df_prova_2012['area'].apply(str) + '_' + df_prova_2012['cor'].apply(str)
df_2012_merge = pd.merge(df_itens_2012, df_prova_2012, on='id', how='left')
for col in df_2012_merge.columns:
    df_2012_merge = df_2012_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2012), '2012_ENEM_COMPLETA')
df_2012_merge.to_csv(file + '.csv', sep = ';', index = False)
df_2012_merge.to_csv(file + '.csv', sep = ';', index = False)
df_2012_merge.head()

### 2013

In [ ]:
#aplicado a 2013
def extrai_questoes_4(dir_, ano, pdf, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf  = re.sub(r'g\d+', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    #txt_pdf = re.sub(r'\*azul75sab\d+\*', '', txt_pdf)
    #txt_pdf = re.sub(r'\*azul25dom\d+\*', '', txt_pdf)
    #txt_pdf = re.sub(r'\*amar25dom\d+\*', '', txt_pdf)
    txt_pdf = re.sub(r'\*((CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))((25)|(75))((dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    #LC - 2º dia | Caderno 7 - AZUL - Página 31
    #CH - 1º dia | Caderno 2 - AMARELO - Página 10
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    #dia | Caderno 2 - AMARELO - Página 18
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    letras = ['\nA ', '\nB ',  '\nC ','\nD ', '\nE ']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if len(questao) == 0:
                pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                questao = pattern.findall(txt_pdf)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print(questao)
        else:
            pattern = re.compile(r"QUESTÃO "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_A', 8: 'alternativa_B', 9: 'alternativa_C', 10: 'alternativa_D', 11: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
file_path, df_Caderno1_Azul_Sab = extrai_questoes_4('_itens', 2013, 'Caderno1_Azul_Sab.pdf', 'AZUL', 'PAPEL', 'CH', 1, 90)

In [ ]:
df_Caderno1_Azul_Sab.loc[df_Caderno1_Azul_Sab['num_questao'] <= 45, 'area'] = 'CH'
df_Caderno1_Azul_Sab.loc[df_Caderno1_Azul_Sab['num_questao'] > 45, 'area'] = 'CN'
print(df_Caderno1_Azul_Sab.area.unique())
df_Caderno1_Azul_Sab.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_Caderno2_Amarelo_Sab = extrai_questoes_4('_itens', 2013, 'Caderno2_Amarelo_Sab.pdf', 'AMARELO', 'PAPEL', 'CH', 1, 90)

In [ ]:
df_Caderno2_Amarelo_Sab.loc[df_Caderno2_Amarelo_Sab['num_questao'] <= 45, 'area'] = 'CH'
df_Caderno2_Amarelo_Sab.loc[df_Caderno2_Amarelo_Sab['num_questao'] > 45, 'area'] = 'CN'
print(df_Caderno2_Amarelo_Sab.area.unique())
df_Caderno2_Amarelo_Sab.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_Caderno4_Rosa_Sab = extrai_questoes_4('_itens', 2013, 'Caderno4_Rosa_Sab.pdf', 'ROSA', 'PAPEL', 'CH', 1, 90)

In [ ]:
df_Caderno4_Rosa_Sab.loc[df_Caderno4_Rosa_Sab['num_questao'] <= 45, 'area'] = 'CH'
df_Caderno4_Rosa_Sab.loc[df_Caderno4_Rosa_Sab['num_questao'] > 45, 'area'] = 'CN'
print(df_Caderno4_Rosa_Sab.area.unique())
df_Caderno4_Rosa_Sab.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_Caderno5_Amarelo_Dom = extrai_questoes_4('_itens', 2013, 'Caderno5_Amarelo_Dom.pdf', 'AMARELO', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_Caderno5_Amarelo_Dom.loc[df_Caderno5_Amarelo_Dom['num_questao'] <= 135, 'area'] = 'LC'
df_Caderno5_Amarelo_Dom.loc[df_Caderno5_Amarelo_Dom['num_questao'] > 135, 'area'] = 'MT'
print(df_Caderno5_Amarelo_Dom.area.unique())
df_Caderno5_Amarelo_Dom.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_Caderno7_Azul_Dom = extrai_questoes_4('_itens', 2013, 'Caderno7_Azul_Dom.pdf', 'AZUL', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_Caderno7_Azul_Dom.loc[df_Caderno7_Azul_Dom['num_questao'] <= 135, 'area'] = 'LC'
df_Caderno7_Azul_Dom.loc[df_Caderno7_Azul_Dom['num_questao'] > 135, 'area'] = 'MT'
print(df_Caderno7_Azul_Dom.area.unique())
df_Caderno7_Azul_Dom.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_Caderno8_Rosa_Dom = extrai_questoes_4('_itens', 2013, 'Caderno8_Rosa_Dom.pdf', 'ROSA', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_Caderno8_Rosa_Dom.loc[df_Caderno8_Rosa_Dom ['num_questao'] <= 135, 'area'] = 'LC'
df_Caderno8_Rosa_Dom.loc[df_Caderno8_Rosa_Dom ['num_questao'] > 135, 'area'] = 'MT'
print(df_Caderno8_Rosa_Dom.area.unique())
df_Caderno8_Rosa_Dom.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2013 = pd.concat([df_Caderno1_Azul_Sab, 
                           df_Caderno2_Amarelo_Sab, 
                           df_Caderno4_Rosa_Sab,
                           df_Caderno5_Amarelo_Dom,
                           df_Caderno8_Rosa_Dom,
                           df_Caderno7_Azul_Dom], ignore_index=True, axis = 0)

# Resultado
df_prova_2013.shape

In [ ]:
file = os.path.join('_itens', str(2013), 'ITENS_PROVA_2013')
df_itens_2013 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')

In [ ]:
df_itens_2013['id'] = df_itens_2013['CO_POSICAO'].apply(str) + '_' + df_itens_2013['SG_AREA'].apply(str) + '_' + df_itens_2013['TX_COR'].apply(str)
df_prova_2013['id'] = df_prova_2013['num_questao'].apply(str) + '_' + df_prova_2013['area'].apply(str) + '_' + df_prova_2013['cor'].apply(str)
df_2013_merge = pd.merge(df_itens_2013, df_prova_2013, on='id', how='left')
for col in df_2013_merge.columns:
    df_2013_merge = df_2013_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2013), '2013_ENEM_COMPLETA')
df_2013_merge.to_csv(file + '.csv', sep = ';', index = False)
df_2013_merge.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_2013_merge.head(60)

### 2014

In [ ]:
#aplicado a 2013
def extrai_questoes_5(dir_, ano, pdf, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf  = re.sub(r'g\d+', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    #txt_pdf = re.sub(r'\*azul75sab\d+\*', '', txt_pdf)
    #txt_pdf = re.sub(r'\*azul25dom\d+\*', '', txt_pdf)
    #txt_pdf = re.sub(r'\*amar25dom\d+\*', '', txt_pdf)
    txt_pdf = re.sub(r'\*((CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))((25)|(75))((dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    txt_pdf = txt_pdf.replace('23456789101112131415161718192021222324252627282930', '')
    
    #LC - 2º dia | Caderno 7 - AZUL - Página 31
    #CH - 1º dia | Caderno 2 - AMARELO - Página 10
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    #dia | Caderno 2 - AMARELO - Página 18
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    txt_pdf = txt_pdf.replace('2014', '')    
    letras = ['\nA ', '\nB ',  '\nC ','\nD ', '\nE ']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            #if len(questao) == 0:
                #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                #questao = pattern.findall(txt_pdf)
            pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print(questao)
        else:
            pattern = re.compile(r"QUESTÃO "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_A', 8: 'alternativa_B', 9: 'alternativa_C', 10: 'alternativa_D', 11: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
file_path, df_2014_PV_impresso_D1_CD1= extrai_questoes_5('_itens', 2014, '2014_PV_impresso_D1_CD1.pdf', 'Azul', 'PAPEL', 'CN', 1, 90)

In [ ]:
df_2014_PV_impresso_D1_CD1.loc[df_2014_PV_impresso_D1_CD1['num_questao'] <= 45, 'area'] = 'CH'
df_2014_PV_impresso_D1_CD1.loc[df_2014_PV_impresso_D1_CD1['num_questao'] > 45, 'area'] = 'CN'
print(df_2014_PV_impresso_D1_CD1.area.unique())
df_2014_PV_impresso_D1_CD1.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_2014_PV_impresso_D2_CD5 = extrai_questoes_5('_itens', 2014, '2014_PV_impresso_D2_CD5.pdf', 'Amarelo', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_2014_PV_impresso_D2_CD5.loc[df_2014_PV_impresso_D2_CD5['num_questao'] <= 135, 'area'] = 'LC'
df_2014_PV_impresso_D2_CD5.loc[df_2014_PV_impresso_D2_CD5['num_questao'] > 135, 'area'] = 'MT'
print(df_2014_PV_impresso_D2_CD5.area.unique())
df_2014_PV_impresso_D2_CD5.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2014 = pd.concat([df_2014_PV_impresso_D1_CD1, 
                           df_2014_PV_impresso_D2_CD5], ignore_index=True, axis = 0)

# Resultado
df_prova_2014.shape

In [ ]:
file = os.path.join('_itens', str(2014), 'ITENS_PROVA_2014')
df_itens_2014 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')

In [ ]:
df_itens_2014['id'] = df_itens_2014['CO_POSICAO'].apply(str) + '_' + df_itens_2014['SG_AREA'].apply(str) + '_' + df_itens_2014['TX_COR'].apply(str)
df_prova_2014['id'] = df_prova_2014['num_questao'].apply(str) + '_' + df_prova_2014['area'].apply(str) + '_' + df_prova_2014['cor'].apply(str)
df_2014_merge = pd.merge(df_itens_2014, df_prova_2014, on='id', how='left')
for col in df_2014_merge.columns:
    df_2014_merge = df_2014_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2014), '2014_ENEM_COMPLETA')
df_2014_merge.to_csv(file + '.csv', sep = ';', index = False)
df_2014_merge.to_csv(file + '.csv', sep = ';', index = False)

### 2015

In [ ]:
#aplicado a 2015
def extrai_questoes_6(dir_, ano, pdf, cor, meio_aplicacao, area, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    txt_pdf = re.sub(r'\*((CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))((25)|(75))((dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    txt_pdf = txt_pdf.replace('2014', '')
    txt_pdf = txt_pdf.replace('\n2015', '')
    txt_pdf = txt_pdf.replace('. 2015', '')
    txt_pdf = txt_pdf.replace('.  2015', '')
    letras = ['\nA ', '\nB ',  '\nC ','\nD ', '\nE ']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            #if len(questao) == 0:
                #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                #questao = pattern.findall(txt_pdf)
            pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = alternativa[0].replace('\n','').strip()
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print(questao)
        else:
            pattern = re.compile(r"QUESTÃO "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        alternativa = alternativa.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        alternativa = alternativa.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            questao = questao.replace('RAScUnHODA REDAÇÃO2015123456789101112131415161718192021222324252627282930', '')
            questao = questao.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'texto_questao', 6: 'texto_base', 7: 'alternativa_A', 8: 'alternativa_B', 9: 'alternativa_C', 10: 'alternativa_D', 11: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

In [ ]:
file_path, df_2015_PV_impresso_D1_CD1 = extrai_questoes_6('_itens', 2015, '2015_PV_impresso_D1_CD1.pdf', 'Azul', 'PAPEL', 'CH', 1, 90)

In [ ]:
df_2015_PV_impresso_D1_CD1.loc[df_2015_PV_impresso_D1_CD1['num_questao'] <= 45, 'area'] = 'CH'
df_2015_PV_impresso_D1_CD1.loc[df_2015_PV_impresso_D1_CD1['num_questao'] > 45, 'area'] = 'CN'
print(df_2015_PV_impresso_D1_CD1.area.unique())
df_2015_PV_impresso_D1_CD1.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_2015_PV_impresso_D2_CD2 = extrai_questoes_6('_itens', 2015, '2015_PV_impresso_D2_CD7.pdf', 'Azul', 'PAPEL', 'LC', 96, 180)

In [ ]:
df_2015_PV_impresso_D2_CD2.loc[df_2015_PV_impresso_D2_CD2['num_questao'] <= 135, 'area'] = 'LC'
df_2015_PV_impresso_D2_CD2.loc[df_2015_PV_impresso_D2_CD2['num_questao'] > 135, 'area'] = 'MT'
print(df_2015_PV_impresso_D2_CD2.area.unique())
df_2015_PV_impresso_D2_CD2.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2015 = pd.concat([df_2015_PV_impresso_D1_CD1, 
                           df_2015_PV_impresso_D2_CD2], ignore_index=True, axis = 0)

# Resultado
df_prova_2015.shape

In [ ]:
file = os.path.join('_itens', str(2015), 'ITENS_PROVA_2015')
df_itens_2015 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2015['id'] = df_itens_2015['CO_POSICAO'].apply(str) + '_' + df_itens_2015['SG_AREA'].apply(str) + '_' + df_itens_2015['TX_COR'].apply(str)
df_prova_2015['id'] = df_prova_2015['num_questao'].apply(str) + '_' + df_prova_2015['area'].apply(str) + '_' + df_prova_2015['cor'].apply(str)
df_2015_merge = pd.merge(df_itens_2015, df_prova_2015, on='id', how='left')
for col in df_2015_merge.columns:
    df_2015_merge = df_2015_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2015), '2015_ENEM_COMPLETA')
df_2015_merge.to_csv(file + '.csv', sep = ';', index = False)
df_2015_merge.to_csv(file + '.csv', sep = ';', index = False)

### 2017

In [ ]:
#aplicado a 2015
def extrai_questoes_7(dir_, ano, pdf, cor, meio_aplicacao, area, co_prova, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    #*SA0175AZ6*
    #*DO0525AM31*
    txt_pdf = re.sub(r"\*2017[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r"\*[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r'\*((SA)(CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))[0-9]+((AZ)(dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    txt_pdf = txt_pdf.replace('2014', '')
    txt_pdf = txt_pdf.replace('\n2015', '')
    txt_pdf = txt_pdf.replace('. 2015', '')
    txt_pdf = txt_pdf.replace('.  2015', '')
    txt_pdf = txt_pdf.replace('- 2017', '')
    txt_pdf = txt_pdf.replace('23456789101112131415161718192021222324252627282930', '')
    letras = ['\nA ', '\nB ', '\nC ', '\nD ', '\nE ']
    #letras = ['A\n ', 'B\n ', 'C\n ', 'D\n ', 'E\n']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            #if len(questao) == 0:
                #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                #questao = pattern.findall(txt_pdf)
            pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    #(?<=(A\n|\nA)).*?(?=B\n|\nB)
                    #pattern = re.compile(r'(?<=(\n'+letras[j]+' |'+letras[j]+'\n|\n'+letras[j]+')).*?(?=\n'+letras[j+1]+' |'+letras[j+1]+'\n|\n'+letras[j+1]+' )', re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print(questao)
        else:
            pattern = re.compile(r"QUESTÃO "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        alternativa = alternativa.replace('23456789101112131415161718192021222324252627282930', '')
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        alternativa = alternativa.replace('23456789101112131415161718192021222324252627282930', '')
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            questao = questao.replace('RAScUnHODA REDAÇÃO2015123456789101112131415161718192021222324252627282930', '')
            questao = questao.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, co_prova, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'co_prova', 6: 'texto_questao', 7: 'texto_base', 8: 'alternativa_A', 9: 'alternativa_B', 10: 'alternativa_C', 11: 'alternativa_D', 12: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

#### P1

In [ ]:
file_path, df_ENEM_2017_P1_CAD_01_DIA_1_AZUL = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_01_DIA_1_AZUL.pdf', 'AZUL', 'PAPEL', 'LC', 399,  6, 90)

In [ ]:
df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'co_prova'] = 399
df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] > 45, 'co_prova'] = 395
df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'num_questao'] = df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'num_questao'] + 5
df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[(df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['area'] == 'CH'), 'num_questao'] = df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.loc[(df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_01_DIA_1_AZUL['area'] == 'CH'), 'num_questao'] - 45
print(np.sort(df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.area.unique())
print(df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.co_prova.unique())
df_ENEM_2017_P1_CAD_01_DIA_1_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_02_DIA_1_AMARELO.pdf', 'AMARELA', 'PAPEL', 'LC', 400,  6, 90)

In [ ]:
df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'co_prova'] = 400
df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'co_prova'] = 396
df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'num_questao'] = df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'num_questao'] + 5
df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[(df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['area'] == 'CH'), 'num_questao'] = df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.loc[(df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO['area'] == 'CH'), 'num_questao'] - 45
print(np.sort(df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.area.unique())
print(df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.co_prova.unique())
df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_03_DIA_1_BRANCO.pdf', 'BRANCA', 'PAPEL', 'LC', 402,  6, 90)

In [ ]:
df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'co_prova'] = 402
df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'co_prova'] = 397
df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'num_questao'] = df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'num_questao'] + 5
df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[(df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['area'] == 'CH'), 'num_questao'] = df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.loc[(df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO['area'] == 'CH'), 'num_questao'] - 45
print(np.sort(df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.area.unique())
print(df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.co_prova.unique())
df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P1_CAD_04_DIA_1_ROSA = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_04_DIA_1_ROSA.pdf', 'ROSA', 'PAPEL', 'LC', 402,  6, 90)

In [ ]:
df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'co_prova'] = 401
df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'co_prova'] = 398
df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'num_questao'] = df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'num_questao'] + 5
df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[(df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['area'] == 'CH'), 'num_questao'] = df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.loc[(df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['num_questao'] > 45) & (df_ENEM_2017_P1_CAD_04_DIA_1_ROSA['area'] == 'CH'), 'num_questao'] - 45
print(np.sort(df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.area.unique())
print(df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.co_prova.unique())
df_ENEM_2017_P1_CAD_04_DIA_1_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_05_DIA_2_AMARELO.pdf', 'AMARELA', 'PAPEL', 'CN', 392,  91, 180)

In [ ]:
df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'co_prova'] = 392
df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'co_prova'] = 404
df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.area.unique())
print(df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.co_prova.unique())
df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P1_CAD_06_DIA_2_CINZA = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_06_DIA_2_CINZA.pdf', 'CINZA', 'PAPEL', 'CN', 393,  91, 180)

In [ ]:
df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'co_prova'] = 393
df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'co_prova'] = 406
df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P1_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.area.unique())
print(df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.co_prova.unique())
df_ENEM_2017_P1_CAD_06_DIA_2_CINZA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P1_CAD_07_DIA_2_AZUL = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_07_DIA_2_AZUL.pdf', 'AZUL', 'PAPEL', 'CN', 391,  91, 180)

In [ ]:
df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'co_prova'] = 391
df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'co_prova'] = 403
df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P1_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.area.unique())
print(df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.co_prova.unique())
df_ENEM_2017_P1_CAD_07_DIA_2_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P1_CAD_08_DIA_2_ROSA = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P1_CAD_08_DIA_2_ROSA.pdf', 'ROSA', 'PAPEL', 'CN', 394,  91, 180)

In [ ]:
df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'co_prova'] = 394
df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'co_prova'] = 405
df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P1_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.num_questao.unique()))
print(df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.area.unique())
print(df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.co_prova.unique())
df_ENEM_2017_P1_CAD_08_DIA_2_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

#### P2

In [ ]:
file_path, df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P2_CAD_02_DIA_1_AMARELO.pdf', 'AMARELA', 'PAPEL', 'LC', 440, 6, 90)

In [ ]:
df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'co_prova'] = 440
df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'co_prova'] = 436
df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'num_questao'] = df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'num_questao'] + 5
df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[(df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] > 45) & (df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['area'] == 'CH'), 'num_questao'] = df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.loc[(df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['num_questao'] > 45) & (df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO['area'] == 'CH'), 'num_questao'] - 45
print(np.sort(df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.num_questao.unique()))
print(df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.area.unique())
print(df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.co_prova.unique())
df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P2_CAD_03_DIA_1_BRANCO.pdf', 'BRANCA', 'PAPEL', 'LC', 441, 6, 90)

In [ ]:
df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'co_prova'] = 441
df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'co_prova'] = 437
df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'num_questao'] = df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'num_questao'] + 5
df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[(df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] > 45) & (df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['area'] == 'CH'), 'num_questao'] = df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.loc[(df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['num_questao'] > 45) & (df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO['area'] == 'CH'), 'num_questao'] - 45
print(np.sort(df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.num_questao.unique()))
print(df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.area.unique())
print(df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.co_prova.unique())
df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P2_CAD_04_DIA_1_ROSA = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P2_CAD_04_DIA_1_ROSA.pdf', 'ROSA', 'PAPEL', 'LC', 442, 6, 90)

In [ ]:
df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'co_prova'] = 442
df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'co_prova'] = 438
df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'num_questao'] = df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'num_questao'] + 5
df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[(df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] > 45) & (df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['area'] == 'CH'), 'num_questao'] = df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.loc[(df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['num_questao'] > 45) & (df_ENEM_2017_P2_CAD_04_DIA_1_ROSA['area'] == 'CH'), 'num_questao'] - 45
print(np.sort(df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.num_questao.unique()))
print(df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.area.unique())
print(df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.co_prova.unique())
df_ENEM_2017_P2_CAD_04_DIA_1_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P2_CAD_05_DIA_2_AMARELO.pdf', 'AMARELA', 'PAPEL', 'CN', 431, 91, 180)

In [ ]:
df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'co_prova'] = 431
df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'co_prova'] = 443
df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.num_questao.unique()))
print(df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.area.unique())
print(df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.co_prova.unique())
df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P2_CAD_06_DIA_2_CINZA = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P2_CAD_06_DIA_2_CINZA.pdf', 'CINZA', 'PAPEL', 'CN', 432, 91, 180)

In [ ]:
df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'co_prova'] = 432
df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'co_prova'] = 444
df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2017_P2_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.num_questao.unique()))
print(df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.area.unique())
print(df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.co_prova.unique())
df_ENEM_2017_P2_CAD_06_DIA_2_CINZA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P2_CAD_07_DIA_2_AZUL = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P2_CAD_07_DIA_2_AZUL.pdf', 'AZUL', 'PAPEL', 'CN', 433, 91, 180)

In [ ]:
df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'co_prova'] = 433
df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'co_prova'] = 445
df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2017_P2_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.num_questao.unique()))
print(df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.area.unique())
print(df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.co_prova.unique())
df_ENEM_2017_P2_CAD_07_DIA_2_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2017_P2_CAD_08_DIA_2_ROSA = extrai_questoes_7('_itens', 2017, 'ENEM_2017_P2_CAD_08_DIA_2_ROSA.pdf', 'ROSA', 'PAPEL', 'CN', 434, 91, 180)

In [ ]:
df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'co_prova'] = 434
df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'co_prova'] = 446
df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'num_questao'] = df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'num_questao'] - 90
df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'num_questao'] = df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2017_P2_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'num_questao'] - 135
print(np.sort(df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.num_questao.unique()))
print(df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.area.unique())
print(df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.co_prova.unique())
df_ENEM_2017_P2_CAD_08_DIA_2_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2017 = pd.concat([df_ENEM_2017_P1_CAD_01_DIA_1_AZUL,
                            df_ENEM_2017_P1_CAD_02_DIA_1_AMARELO,
                            df_ENEM_2017_P1_CAD_03_DIA_1_BRANCO,
                            df_ENEM_2017_P1_CAD_04_DIA_1_ROSA,
                            df_ENEM_2017_P1_CAD_05_DIA_2_AMARELO,
                            df_ENEM_2017_P1_CAD_06_DIA_2_CINZA,
                            df_ENEM_2017_P1_CAD_07_DIA_2_AZUL,
                            df_ENEM_2017_P1_CAD_08_DIA_2_ROSA,
                            df_ENEM_2017_P2_CAD_02_DIA_1_AMARELO,
                            df_ENEM_2017_P2_CAD_03_DIA_1_BRANCO,
                            df_ENEM_2017_P2_CAD_04_DIA_1_ROSA,
                            df_ENEM_2017_P2_CAD_05_DIA_2_AMARELO,
                            df_ENEM_2017_P2_CAD_06_DIA_2_CINZA,
                            df_ENEM_2017_P2_CAD_07_DIA_2_AZUL,
                            df_ENEM_2017_P2_CAD_08_DIA_2_ROSA], ignore_index=True, axis = 0)

# Resultado
df_prova_2017.shape

In [ ]:
file = os.path.join('_itens', str(2017), 'ITENS_PROVA_2017')
df_itens_2017 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2017['id'] = df_itens_2017['CO_POSICAO'].apply(str) + '_' + df_itens_2017['SG_AREA'].apply(str) + '_' + df_itens_2017['TX_COR'].apply(str) + '_' + df_itens_2017['CO_PROVA'].apply(str)
df_prova_2017['id'] = df_prova_2017['num_questao'].apply(str) + '_' + df_prova_2017['area'].apply(str) + '_' + df_prova_2017['cor'].apply(str) + '_' + df_prova_2017['co_prova'].apply(str)
df_2017_merge = pd.merge(df_itens_2017, df_prova_2017, on='id', how='left')
df_2017_merge = df_2017_merge.drop(df_2017_merge.columns[20], axis = 1)
for col in df_2017_merge.columns:
    df_2017_merge = df_2017_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2017), '2017_ENEM_COMPLETA')
df_2017_merge.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_2017_merge.head()

### 2019

In [ ]:
#aplicado a 2019
def extrai_questoes_8(dir_, ano, pdf, cor, meio_aplicacao, area, co_prova, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    #*SA0175AZ6*
    #*DO0525AM31*
    txt_pdf = re.sub(r"\*2017[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r"\*[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r'\*((SA)(CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))[0-9]+((AZ)(dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    txt_pdf = txt_pdf.replace('2014', '')
    txt_pdf = txt_pdf.replace('\n2015', '')
    txt_pdf = txt_pdf.replace('. 2015', '')
    txt_pdf = txt_pdf.replace('.  2015', '')
    txt_pdf = txt_pdf.replace('- 2017', '')
    txt_pdf = txt_pdf.replace('23456789101112131415161718192021222324252627282930', '')
    txt_pdf = txt_pdf.replace('RASCUNH\n DA REDAÇÃO 1', '')
    letras = ['\nA ', '\nB ', '\nC ', '\nD ', '\nE ']
    #letras = ['A\n ', 'B\n ', 'C\n ', 'D\n ', 'E\n']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            #if len(questao) == 0:
                #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                #questao = pattern.findall(txt_pdf)
            pattern = re.compile(r"Questão "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)Questão \d+", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            print('texto_base')
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    #(?<=(A\n|\nA)).*?(?=B\n|\nB)
                    #pattern = re.compile(r'(?<=(\n'+letras[j]+' |'+letras[j]+'\n|\n'+letras[j]+')).*?(?=\n'+letras[j+1]+' |'+letras[j+1]+'\n|\n'+letras[j+1]+' )', re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print('questão')
            print(questao)
        else:
            pattern = re.compile(r"Questão "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                
                print(texto_base)
            else:
                texto_base = ''
                
            print('alternativas')
            for j in range(len(letras)):
                if j < 4:
                    
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        alternativa = alternativa.replace('23456789101112131415161718192021222324252627282930', '')
                        alternativa = alternativa.replace('RASCUNHO DA REDAÇÃO 1', '')
                        alternativa = alternativa.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            questao = questao.replace('RAScUnHODA REDAÇÃO2015123456789101112131415161718192021222324252627282930', '')
            questao = questao.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
            questao = questao.replace('RASCUNHO DA REDAÇÃO 1', '')
            questao = questao.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, co_prova, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'co_prova', 6: 'texto_questao', 7: 'texto_base', 8: 'alternativa_A', 9: 'alternativa_B', 10: 'alternativa_C', 11: 'alternativa_D', 12: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

#### P1

In [ ]:
file_path, df_ENEM_2019_P1_CAD_01_DIA_1_AZUL = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_01_DIA_1_AZUL.pdf', 'Azul', 'PAPEL', 'LC', 511,  6, 90)

In [ ]:
df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P1_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P1_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'co_prova'] = 511
df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P1_CAD_01_DIA_1_AZUL['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P1_CAD_01_DIA_1_AZUL['num_questao'] > 45, 'co_prova'] = 507
print(np.sort(df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.area.unique())
print(df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.co_prova.unique())
df_ENEM_2019_P1_CAD_01_DIA_1_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_02_DIA_1_AMARELO.pdf', 'Amarela', 'PAPEL', 'LC', 512,  6, 90)

In [ ]:
df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'co_prova'] = 512
df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'co_prova'] = 508
print(np.sort(df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.area.unique())
print(df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.co_prova.unique())
df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_03_DIA_1_BRANCO.pdf', 'Branca', 'PAPEL', 'LC', 514,  6, 90)

In [ ]:
df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'co_prova'] = 514
df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'co_prova'] = 509
print(np.sort(df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.area.unique())
print(df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.co_prova.unique())
df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P1_CAD_04_DIA_1_ROSA = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_04_DIA_1_ROSA.pdf', 'Rosa', 'PAPEL', 'LC', 513,  6, 90)

In [ ]:
df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P1_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P1_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'co_prova'] = 513
df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P1_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P1_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'co_prova'] = 510
print(np.sort(df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.area.unique())
print(df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.co_prova.unique())
df_ENEM_2019_P1_CAD_04_DIA_1_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_05_DIA_2_AMARELO.pdf', 'Amarela', 'PAPEL', 'CN', 504,  91, 180)

In [ ]:
df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'co_prova'] = 504
df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'co_prova'] = 516
print(np.sort(df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.area.unique())
print(df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.co_prova.unique())
df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P1_CAD_06_DIA_2_CINZA = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_06_DIA_2_CINZA.pdf', 'Cinza', 'PAPEL', 'CN', 505,  91, 180)

In [ ]:
df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P1_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P1_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'co_prova'] = 505
df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P1_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P1_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'co_prova'] = 518
print(np.sort(df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.area.unique())
print(df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.co_prova.unique())
df_ENEM_2019_P1_CAD_06_DIA_2_CINZA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P1_CAD_07_DIA_2_AZUL = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_07_DIA_2_AZUL.pdf', 'Azul', 'PAPEL', 'CN', 503,  91, 180)

In [ ]:
df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P1_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P1_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'co_prova'] = 503
df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P1_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P1_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'co_prova'] = 515
print(np.sort(df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.area.unique())
print(df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.co_prova.unique())
df_ENEM_2019_P1_CAD_07_DIA_2_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P1_CAD_08_DIA_2_ROSA = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P1_CAD_08_DIA_2_ROSA.pdf', 'Rosa', 'PAPEL', 'CN', 506,  91, 180)

In [ ]:
df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P1_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P1_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'co_prova'] = 506
df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P1_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P1_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'co_prova'] = 517
print(np.sort(df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.num_questao.unique()))
print(df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.area.unique())
print(df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.co_prova.unique())
df_ENEM_2019_P1_CAD_08_DIA_2_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

#### P2

In [ ]:
file_path, df_ENEM_2019_P2_CAD_01_DIA_1_AZUL = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_01_DIA_1_AZUL.pdf', 'Azul', 'PAPEL', 'LC', 551,  6, 90)

In [ ]:
df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P2_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P2_CAD_01_DIA_1_AZUL['num_questao'] <= 45, 'co_prova'] = 551
df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P2_CAD_01_DIA_1_AZUL['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.loc[df_ENEM_2019_P2_CAD_01_DIA_1_AZUL['num_questao'] > 45, 'co_prova'] = 547
print(np.sort(df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.area.unique())
print(df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.co_prova.unique())
df_ENEM_2019_P2_CAD_01_DIA_1_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_02_DIA_1_AMARELO.pdf', 'Amarela', 'PAPEL', 'LC', 552,  6, 90)

In [ ]:
df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO['num_questao'] <= 45, 'co_prova'] = 552
df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.loc[df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO['num_questao'] > 45, 'co_prova'] = 548
print(np.sort(df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.area.unique())
print(df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.co_prova.unique())
df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P2_CAD_04_DIA_1_ROSA = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_04_DIA_1_ROSA.pdf', 'Rosa', 'PAPEL', 'LC', 550,  6, 90)

In [ ]:
df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P2_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P2_CAD_04_DIA_1_ROSA['num_questao'] <= 45, 'co_prova'] = 554
df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P2_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.loc[df_ENEM_2019_P2_CAD_04_DIA_1_ROSA['num_questao'] > 45, 'co_prova'] = 550
print(np.sort(df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.area.unique())
print(df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.co_prova.unique())
df_ENEM_2019_P2_CAD_04_DIA_1_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_03_DIA_1_BRANCO.pdf', 'Branca', 'PAPEL', 'LC', 553,  6, 90)

In [ ]:
df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO['num_questao'] <= 45, 'co_prova'] = 553
df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.loc[df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO['num_questao'] > 45, 'co_prova'] = 549
print(np.sort(df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.area.unique())
print(df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.co_prova.unique())
df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_05_DIA_2_AMARELO.pdf', 'Amarela', 'PAPEL', 'LC', 552,  91, 180)

In [ ]:
df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'co_prova'] = 552
df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'co_prova'] = 548
print(np.sort(df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.area.unique())
print(df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.co_prova.unique())
df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P2_CAD_06_DIA_2_CINZA = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_06_DIA_2_CINZA.pdf', 'Cinza', 'PAPEL', 'CN', 544,  91, 180)

In [ ]:
df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P2_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P2_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'co_prova'] = 544
df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P2_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.loc[df_ENEM_2019_P2_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'co_prova'] = 556
print(np.sort(df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.area.unique())
print(df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.co_prova.unique())
df_ENEM_2019_P2_CAD_06_DIA_2_CINZA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P2_CAD_07_DIA_2_AZUL = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_07_DIA_2_AZUL.pdf', 'Azul', 'PAPEL', 'CN', 545,  91, 180)

In [ ]:
df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P2_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P2_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'co_prova'] = 545
df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P2_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.loc[df_ENEM_2019_P2_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'co_prova'] = 557
print(np.sort(df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.area.unique())
print(df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.co_prova.unique())
df_ENEM_2019_P2_CAD_07_DIA_2_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2019_P2_CAD_08_DIA_2_ROSA = extrai_questoes_8('_itens', 2019, 'ENEM_2019_P2_CAD_08_DIA_2_ROSA.pdf', 'Rosa', 'PAPEL', 'CN', 546,  91, 180)

In [ ]:
df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P2_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P2_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'co_prova'] = 546
df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P2_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.loc[df_ENEM_2019_P2_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'co_prova'] = 558
print(np.sort(df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.num_questao.unique()))
print(df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.area.unique())
print(df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.co_prova.unique())
df_ENEM_2019_P2_CAD_08_DIA_2_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
# Concatenação dos dataframes de 2009
df_prova_2019 = pd.concat([df_ENEM_2019_P1_CAD_01_DIA_1_AZUL,
                            df_ENEM_2019_P1_CAD_02_DIA_1_AMARELO,
                            df_ENEM_2019_P1_CAD_03_DIA_1_BRANCO,
                            df_ENEM_2019_P1_CAD_04_DIA_1_ROSA,
                            df_ENEM_2019_P1_CAD_05_DIA_2_AMARELO,
                            df_ENEM_2019_P1_CAD_06_DIA_2_CINZA,
                            df_ENEM_2019_P1_CAD_07_DIA_2_AZUL,
                            df_ENEM_2019_P1_CAD_08_DIA_2_ROSA,
                            df_ENEM_2019_P2_CAD_01_DIA_1_AZUL,
                            df_ENEM_2019_P2_CAD_02_DIA_1_AMARELO,
                            df_ENEM_2019_P2_CAD_03_DIA_1_BRANCO,
                            df_ENEM_2019_P2_CAD_04_DIA_1_ROSA,
                            df_ENEM_2019_P2_CAD_05_DIA_2_AMARELO,
                            df_ENEM_2019_P2_CAD_06_DIA_2_CINZA,
                            df_ENEM_2019_P2_CAD_07_DIA_2_AZUL,
                            df_ENEM_2019_P2_CAD_08_DIA_2_ROSA], ignore_index=True, axis = 0)

# Resultado
df_prova_2019.shape

In [ ]:
file = os.path.join('_itens', str(2019), 'ITENS_PROVA_2019')
df_itens_2019 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2019['id'] = df_itens_2019['CO_POSICAO'].apply(str) + '_' + df_itens_2019['SG_AREA'].apply(str) + '_' + df_itens_2019['TX_COR'].apply(str) + '_' + df_itens_2019['CO_PROVA'].apply(str)
df_prova_2019['id'] = df_prova_2019['num_questao'].apply(str) + '_' + df_prova_2019['area'].apply(str) + '_' + df_prova_2019['cor'].apply(str) + '_' + df_prova_2019['co_prova'].apply(str)
df_2019_merge = pd.merge(df_itens_2019, df_prova_2019, on='id', how='left')
df_2019_merge = df_2019_merge.drop(df_2019_merge.columns[20], axis = 1)
for col in df_2019_merge.columns:
    df_2019_merge = df_2019_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2019), '2019_ENEM_COMPLETA')
df_2019_merge.to_csv(file + '.csv', sep = ';', index = False)

### 2020

In [ ]:
#aplicado a 2020
def extrai_questoes_9(dir_, ano, pdf, cor, meio_aplicacao, area, co_prova, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    #*SA0175AZ6*
    #*DO0525AM31*
    txt_pdf = re.sub(r"\*2017[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r"\*[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r'\*((SA)(CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))[0-9]+((AZ)(dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    txt_pdf = txt_pdf.replace('2014', '')
    txt_pdf = txt_pdf.replace('\n2015', '')
    txt_pdf = txt_pdf.replace('. 2015', '')
    txt_pdf = txt_pdf.replace('.  2015', '')
    txt_pdf = txt_pdf.replace('- 2017', '')
    txt_pdf = txt_pdf.replace('23456789101112131415161718192021222324252627282930', '')
    txt_pdf = txt_pdf.replace('RASCUNH\n DA REDAÇÃO 1', '')
    txt_pdf = txt_pdf.replace(' - Linguagens, Códigos e suas Tecnologias', '')
    txt_pdf = txt_pdf.replace(' - Ciências Humanas e suas Tecnologias', '')
    letras = ['\nA ', '\nB ', '\nC ', '\nD ', '\nE ']
    #letras = ['A\n ', 'B\n ', 'C\n ', 'D\n ', 'E\n']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            #if len(questao) == 0:
                #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                #questao = pattern.findall(txt_pdf)
            pattern = re.compile(r"Questão "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)Questão \d+", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            print('texto_base')
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    #(?<=(A\n|\nA)).*?(?=B\n|\nB)
                    #pattern = re.compile(r'(?<=(\n'+letras[j]+' |'+letras[j]+'\n|\n'+letras[j]+')).*?(?=\n'+letras[j+1]+' |'+letras[j+1]+'\n|\n'+letras[j+1]+' )', re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print('questão')
            print(questao)
        else:
            pattern = re.compile(r"Questão "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                
                print(texto_base)
            else:
                texto_base = ''
                
            print('alternativas')
            for j in range(len(letras)):
                if j < 4:
                    
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        alternativa = alternativa.replace('23456789101112131415161718192021222324252627282930', '')
                        alternativa = alternativa.replace('RASCUNHO DA REDAÇÃO 1', '')
                        alternativa = alternativa.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            questao = questao.replace('RAScUnHODA REDAÇÃO2015123456789101112131415161718192021222324252627282930', '')
            questao = questao.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
            questao = questao.replace('RASCUNHO DA REDAÇÃO 1', '')
            questao = questao.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, area, co_prova, questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'co_prova', 6: 'texto_questao', 7: 'texto_base', 8: 'alternativa_A', 9: 'alternativa_B', 10: 'alternativa_C', 11: 'alternativa_D', 12: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)
    
    return file_path, df_dados

#### DIGITAL

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.pdf', 'AZUL', 1, 'LC', 691,  6, 90)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES['num_questao'] <= 45, 'co_prova'] = 691
df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES['num_questao'] > 45, 'co_prova'] = 687
print(np.sort(df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.pdf', 'AMARELA', 1, 'LC', 692,  6, 90)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES['num_questao'] <= 45, 'co_prova'] = 692
df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES['num_questao'] > 45, 'co_prova'] = 688
print(np.sort(df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.pdf', 'BRANCA', 1, 'LC', 693,  6, 90)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES['num_questao'] <= 45, 'co_prova'] = 693
df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES['num_questao'] > 45, 'co_prova'] = 689
print(np.sort(df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.pdf', 'ROSA', 1, 'LC', 694,  6, 90)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES['num_questao'] <= 45, 'area'] = 'LC'
df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES['num_questao'] <= 45, 'co_prova'] = 694
df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES['num_questao'] > 45, 'area'] = 'CH'
df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.loc[df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES['num_questao'] > 45, 'co_prova'] = 690
print(np.sort(df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.pdf', 'AMARELA', 1, 'CN', 700,  91, 180)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO['num_questao'] <= 135, 'co_prova'] = 700
df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.loc[df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO['num_questao'] > 135, 'co_prova'] = 696
print(np.sort(df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.pdf', 'CINZA', 1, 'CN', 702,  91, 180)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.loc[df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.loc[df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA['num_questao'] <= 135, 'co_prova'] = 702
df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.loc[df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.loc[df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA['num_questao'] > 135, 'co_prova'] = 698
print(np.sort(df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.pdf', 'AZUL', 1, 'CN', 699,  91, 180)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.loc[df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.loc[df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL['num_questao'] <= 135, 'co_prova'] = 699
df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.loc[df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.loc[df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL['num_questao'] > 135, 'co_prova'] = 695
print(np.sort(df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
file_path, df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA = extrai_questoes_9('_itens', 2020, 'ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.pdf', 'ROSA', 1, 'CN', 701,  91, 180)

In [ ]:
df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.loc[df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'area'] = 'CN'
df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.loc[df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA['num_questao'] <= 135, 'co_prova'] = 701
df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.loc[df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'area'] = 'MT'
df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.loc[df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA['num_questao'] > 135, 'co_prova'] = 697
print(np.sort(df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.num_questao.unique()))
print(df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.area.unique())
print(df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.co_prova.unique())
df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA.to_csv(file_path + '.csv', sep = ';', index = False)

In [ ]:
#aplicado a 2020
def extrai_questoes_10(dir_, ano, pdf, cor, meio_aplicacao, areas, cos_prova, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    #*SA0175AZ6*
    #*DO0525AM31*
    #*010175AZ22*
    txt_pdf = re.sub(r"\*2017[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r"\*[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r"\*\d{6}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r'\*((SA)(CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))[0-9]+((AZ)(dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    txt_pdf = txt_pdf.replace('2014', '')
    txt_pdf = txt_pdf.replace('\n2015', '')
    txt_pdf = txt_pdf.replace('. 2015', '')
    txt_pdf = txt_pdf.replace('.  2015', '')
    txt_pdf = txt_pdf.replace('- 2017', '')
    txt_pdf = txt_pdf.replace('23456789101112131415161718192021222324252627282930', '')
    txt_pdf = txt_pdf.replace('RASCUNH\n DA REDAÇÃO 1', '')
    txt_pdf = txt_pdf.replace(' - Linguagens, Códigos e suas Tecnologias', '')
    txt_pdf = txt_pdf.replace(' - Ciências Humanas e suas Tecnologias', '')
    txt_pdf = txt_pdf.replace('CH - 1°', '')
    txt_pdf = txt_pdf.replace('- 1°', '')
    txt_pdf = txt_pdf.replace('- 2°', '')
    txt_pdf = txt_pdf.replace('2ª aplicação', '')
    letras = ['\nA ', '\nB ', '\nC ', '\nD ', '\nE ']
    #letras = ['A\n ', 'B\n ', 'C\n ', 'D\n ', 'E\n']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            #if len(questao) == 0:
                #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                #questao = pattern.findall(txt_pdf)
            #pattern = re.compile(r"Questão "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)Questão \d+", re.DOTALL)
            pattern = re.compile(r"Questão "+str(i)+"(.*?)Questão \d+", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            print('texto_base')
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    #(?<=(A\n|\nA)).*?(?=B\n|\nB)
                    #pattern = re.compile(r'(?<=(\n'+letras[j]+' |'+letras[j]+'\n|\n'+letras[j]+')).*?(?=\n'+letras[j+1]+' |'+letras[j+1]+'\n|\n'+letras[j+1]+' )', re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print('questão')
            print(questao)
        else:
            pattern = re.compile(r"Questão "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                
                print(texto_base)
            else:
                texto_base = ''
                
            print('alternativas')
            for j in range(len(letras)):
                if j < 4:
                    
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        alternativa = alternativa.replace('23456789101112131415161718192021222324252627282930', '')
                        alternativa = alternativa.replace('RASCUNHO DA REDAÇÃO 1', '')
                        alternativa = alternativa.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
                        alternativa = alternativa.replace('0101 4 9 10321 1°', '')
                        alternativa = alternativa.replace('0102 1° 9 10321', '')
                        alternativa =  alternativa.replace('1° 9 10321 0104', '')
                        alternativa =  alternativa.replace('2° 0201 56789101112', '')
                        alternativa =  alternativa.replace('2° 0203 56789101112', '')
                        alternativa =  alternativa.replace('2° 0202 56789101112', '')
                        alternativa =  alternativa.replace('0204 56789101112 2°', '')
                        alternativa =  alternativa.replace('2ª aplicação 2ª aplicação', '')
                        alternativa =  alternativa.replace('1° 0103  4 9 10321', '')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            questao = questao.replace('RAScUnHODA REDAÇÃO2015123456789101112131415161718192021222324252627282930', '')
            questao = questao.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
            questao = questao.replace('RASCUNHO DA REDAÇÃO 1', '')
            questao = questao.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
            questao = questao.replace('0101 4 9 10321 1°', '')
            questao =  questao.replace('0102 1° 9 10321', '')
            questao =  questao.replace('1° 9 10321 0104', '')
            questao =  questao.replace('2° 0201 56789101112', '')
            questao =  questao.replace('2° 0203 56789101112', '')
            questao =  questao.replace('2° 0202 56789101112', '')
            questao =  questao.replace('0204 56789101112 2°', '')
            questao =  questao.replace('2ª aplicação 2ª aplicação', '')
            questao =  questao.replace('1° 0103  4 9 10321', '')
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, areas[0], cos_prova[0],  questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'co_prova', 6: 'texto_questao', 7: 'texto_base', 8: 'alternativa_A', 9: 'alternativa_B', 10: 'alternativa_C', 11: 'alternativa_D', 12: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    if (areas[0] == 'LC'):
        df_dados.loc[df_dados['num_questao'] <= 45, 'area'] = areas[0]
        df_dados.loc[df_dados['num_questao'] <= 45, 'co_prova'] = cos_prova[0]
        df_dados.loc[df_dados['num_questao'] > 45, 'area'] = areas[1]
        df_dados.loc[df_dados['num_questao'] > 45, 'co_prova'] = cos_prova[1]
        print(np.sort(df_dados.num_questao.unique()))
        print(df_dados.area.unique())
        print(df_dados.co_prova.unique())
    elif (areas[0] == 'CN'):
        df_dados.loc[df_dados['num_questao'] <= 135, 'area'] = areas[0]
        df_dados.loc[df_dados['num_questao'] <= 135, 'co_prova'] = cos_prova[0]
        df_dados.loc[df_dados['num_questao'] > 135, 'area'] = areas[1]
        df_dados.loc[df_dados['num_questao'] > 135, 'co_prova'] = cos_prova[1]
        print(np.sort(df_dados.num_questao.unique()))
        print(df_dados.area.unique())
        print(df_dados.co_prova.unique())
    
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)

    return file_path, df_dados

#### PAPEL - P1

In [ ]:
areas_cos_prova = []

In [ ]:
file_path, df_ENEM_2020_P1_CAD_01_DIA_1_AZUL = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_01_DIA_1_AZUL.pdf', 'AZUL', 0, ['LC', 'CH'], [577, 567],  6, 90)

In [ ]:
file_path, df_ENEM_2020_P1_CAD_02_DIA_1_AMARELO = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_02_DIA_1_AMARELO.pdf', 'AMARELA', 0, ['LC', 'CH'], [578, 568],  6, 90)

In [ ]:
file_path, df_ENEM_2020_P1_CAD_03_DIA_1_BRANCO = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_03_DIA_1_BRANCO.pdf', 'BRANCA', 0, ['LC', 'CH'], [580, 569],  6, 90)

In [ ]:
file_path, df_ENEM_2020_P1_CAD_04_DIA_1_ROSA = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_04_DIA_1_ROSA.pdf', 'ROSA', 0, ['LC', 'CH'], [579, 570],  6, 90)

In [ ]:
file_path, df_ENEM_2020_P1_CAD_05_DIA_2_AMARELO = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_05_DIA_2_AMARELO.pdf', 'AMARELA', 0, ['CN', 'MT'], [598, 588],  91, 180)

In [ ]:
file_path, df_ENEM_2020_P1_CAD_07_DIA_2_AZUL = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_07_DIA_2_AZUL.pdf', 'AZUL', 0, ['CN', 'MT'], [597, 587],  91, 180)

In [ ]:

file_path, df_ENEM_2020_P1_CAD_06_DIA_2_CINZA = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_06_DIA_2_CINZA.pdf', 'CINZA', 0, ['CN', 'MT'], [599, 590],  91, 180)

In [ ]:
file_path, df_ENEM_2020_P1_CAD_08_DIA_2_ROSA = extrai_questoes_10('_itens', 2020, 'ENEM_2020_P1_CAD_08_DIA_2_ROSA.pdf', 'ROSA', 0, ['CN', 'MT'], [600, 589],  91, 180)

#### PAPEL - P2

In [ ]:
dados = [['ENEM_2020_P2_CAD_01_DIA_1_AZUL.pdf', 'AZUL', ['LC', 'CH'], [617, 607], [6, 90]],
        ['ENEM_2020_P2_CAD_02_DIA_1_AMARELO.pdf', 'AMARELA' , ['LC', 'CH'],[618, 608], [6, 90]],
        ['ENEM_2020_P2_CAD_03_DIA_1_BRANCO.pdf', 'BRANCA' , ['LC', 'CH'],[620, 609], [6, 90]],
        ['ENEM_2020_P2_CAD_04_DIA_1_ROSA.pdf', 'ROSA' , ['LC', 'CH'],[619, 610], [6, 90]],
        ['ENEM_2020_P2_CAD_05_DIA_2_AMARELO.pdf', 'AMARELA' , ['CN', 'MT'],[638, 628], [91, 180]],
        ['ENEM_2020_P2_CAD_06_DIA_2_CINZA.pdf', 'CINZA' , ['CN', 'MT'],[639, 630], [91, 180]],
        ['ENEM_2020_P2_CAD_07_DIA_2_AZUL.pdf', 'AZUL' , ['CN', 'MT'],[637, 627], [91, 180]],
        ['ENEM_2020_P2_CAD_08_DIA_2_ROSA.pdf', 'ROSA' , ['CN', 'MT'],[640, 629], [91, 180]]]
dados

In [ ]:
file_path, df_ENEM_2020_P2_CAD_01_DIA_1_AZUL = extrai_questoes_10('_itens', 2020, dados[0][0], dados[0][1], 0, [dados[0][2][0], dados[0][2][1]], [dados[0][3][0], dados[0][3][1]],  dados[0][4][0], dados[0][4][1])

In [ ]:
file_path, df_ENEM_2020_P2_CAD_02_DIA_1_AMARELO = extrai_questoes_10('_itens', 2020, dados[1][0], dados[1][1], 0, [dados[1][2][0], dados[1][2][1]], [dados[1][3][0], dados[1][3][1]],  dados[1][4][0], dados[1][4][1])

In [ ]:
file_path, df_ENEM_2020_P2_CAD_03_DIA_1_BRANCO = extrai_questoes_10('_itens', 2020, dados[2][0], dados[2][1], 0, [dados[2][2][0], dados[2][2][1]], [dados[2][3][0], dados[2][3][1]],  dados[2][4][0], dados[2][4][1])

In [ ]:
file_path, df_ENEM_2020_P2_CAD_04_DIA_1_ROSA = extrai_questoes_10('_itens', 2020, dados[3][0], dados[3][1], 0, [dados[3][2][0], dados[3][2][1]], [dados[3][3][0], dados[3][3][1]],  dados[3][4][0], dados[3][4][1])

In [ ]:
file_path, df_ENEM_2020_P2_CAD_05_DIA_2_AMARELO = extrai_questoes_10('_itens', 2020, dados[4][0], dados[4][1], 0, [dados[4][2][0], dados[4][2][1]], [dados[4][3][0], dados[4][3][1]],  dados[4][4][0], dados[4][4][1])

In [ ]:
file_path, df_ENEM_2020_P2_CAD_06_DIA_2_CINZA = extrai_questoes_10('_itens', 2020, dados[5][0], dados[5][1], 0, [dados[5][2][0], dados[5][2][1]], [dados[5][3][0], dados[5][3][1]],  dados[5][4][0], dados[5][4][1])

In [ ]:
file_path, df_ENEM_2020_P2_CAD_07_DIA_2_AZUL = extrai_questoes_10('_itens', 2020, dados[6][0], dados[6][1], 0, [dados[6][2][0], dados[6][2][1]], [dados[6][3][0], dados[6][3][1]],  dados[6][4][0], dados[6][4][1])

In [ ]:
file_path, df_ENEM_2020_P2_CAD_08_DIA_2_ROSA = extrai_questoes_10('_itens', 2020, dados[7][0], dados[7][1], 0, [dados[7][2][0], dados[7][2][1]], [dados[7][3][0], dados[7][3][1]],  dados[7][4][0], dados[7][4][1])

In [ ]:
# Concatenação dos dataframes de 2020
df_prova_2020 = pd.concat([df_ENEM_2020_DIGITAL_CAD_01_DIA_1_AZUL_INGLES,
                          df_ENEM_2020_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES,
                          df_ENEM_2020_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES,
                          df_ENEM_2020_DIGITAL_CAD_04_DIA_1_ROSA_INGLES,
                          df_ENEM_2020_DIGITAL_CAD_05_DIA_2_AMARELO,
                          df_ENEM_2020_DIGITAL_CAD_06_DIA_2_CINZA,
                          df_ENEM_2020_DIGITAL_CAD_07_DIA_2_AZUL,
                          df_ENEM_2020_DIGITAL_CAD_08_DIA_2_ROSA,
                          df_ENEM_2020_P1_CAD_01_DIA_1_AZUL,
                          df_ENEM_2020_P1_CAD_02_DIA_1_AMARELO,
                          df_ENEM_2020_P1_CAD_03_DIA_1_BRANCO,
                          df_ENEM_2020_P1_CAD_04_DIA_1_ROSA,
                          df_ENEM_2020_P1_CAD_05_DIA_2_AMARELO,
                          df_ENEM_2020_P1_CAD_06_DIA_2_CINZA,
                          df_ENEM_2020_P1_CAD_07_DIA_2_AZUL,
                          df_ENEM_2020_P1_CAD_08_DIA_2_ROSA,
                          df_ENEM_2020_P2_CAD_01_DIA_1_AZUL,
                          df_ENEM_2020_P2_CAD_02_DIA_1_AMARELO,
                          df_ENEM_2020_P2_CAD_03_DIA_1_BRANCO,
                          df_ENEM_2020_P2_CAD_04_DIA_1_ROSA,
                          df_ENEM_2020_P2_CAD_05_DIA_2_AMARELO,
                          df_ENEM_2020_P2_CAD_06_DIA_2_CINZA,
                          df_ENEM_2020_P2_CAD_07_DIA_2_AZUL,
                          df_ENEM_2020_P2_CAD_08_DIA_2_ROSA], ignore_index=True, axis = 0)

# Resultado
df_prova_2020.shape

In [ ]:
file = os.path.join('_itens', str(2020), 'ITENS_PROVA_2020')
df_itens_2020 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2020['id'] = df_itens_2020['CO_POSICAO'].apply(str) + '_' + df_itens_2020['SG_AREA'].apply(str) + '_' + df_itens_2020['TX_COR'].apply(str) + '_' + df_itens_2020['CO_PROVA'].apply(str) + '_' + df_itens_2020['TP_VERSAO_DIGITAL'].apply(str) 
df_prova_2020['id'] = df_prova_2020['num_questao'].apply(str) + '_' + df_prova_2020['area'].apply(str) + '_' + df_prova_2020['cor'].apply(str) + '_' + df_prova_2020['co_prova'].apply(str) + '_' + df_prova_2020['meio_aplicacao'].apply(str) 
df_2020_merge = pd.merge(df_itens_2020, df_prova_2020, on='id', how='left')
df_2020_merge = df_2020_merge.drop(df_2020_merge.columns[20], axis = 1)
for col in df_2020_merge.columns:
    df_2020_merge = df_2020_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2020), '2020_ENEM_COMPLETA')
df_2020_merge.to_csv(file + '.csv', sep = ';', index = False)

### 2021

In [ ]:
file_path = os.path.join('_itens', '2021')
files = os.listdir(file_path)

for file in files:
    print('-', file)

- ENEM_2021_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.pdf LC 889 CH 879
- ENEM_2021_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.pdf LC 890 CH 880
- ENEM_2021_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.pdf LC 892 CH 881
- ENEM_2021_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.pdf LC 891 CH 882
- ENEM_2021_DIGITAL_CAD_05_DIA_2_AMARELO.pdf CN 910 MT 900
- ENEM_2021_DIGITAL_CAD_06_DIA_2_CINZA.pdf CN 911 MT 902
- ENEM_2021_DIGITAL_CAD_07_DIA_2_AZUL.pdf CN 909 MT 899
- ENEM_2021_DIGITAL_CAD_08_DIA_2_ROSA.pdf CN 912 MT 901
- ENEM_2021_P2_CAD_01_DIA_1_AZUL.pdf LC 969 CH 959
- ENEM_2021_P2_CAD_02_DIA_1_AMARELO.pdf LC 970  CH 960
- ENEM_2021_P2_CAD_03_DIA_1_BRANCO.pdf LC 972 CH 961
- ENEM_2021_P2_CAD_04_DIA_1_ROSA.pdf LC 971 CH 962
- ENEM_2021_P2_CAD_05_DIA_2_AMARELO.pdf CN 990 MT 980
- ENEM_2021_P2_CAD_06_DIA_2_CINZA.pdf CN 991 MT 982
- ENEM_2021_P2_CAD_07_DIA_2_AZUL.pdf CN 989 MT 979
- ENEM_2021_P2_CAD_08_DIA_2_ROSA.pdf CN 992 MT 981

In [ ]:
#aplicado a 2021
def extrai_questoes_11(dir_, ano, pdf, cor, meio_aplicacao, areas, cos_prova, num_primeira_questao, num_ultima_questao):
    dados = []
    #arquivo pdf
    file_path = os.path.join(dir_, str(ano), pdf)
    # Abre o arquivo PDF
    pdf_file = open(file_path, "rb")

    # Cria um objeto PDFReader
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    txt_pdf = ''
    # Loop através de cada página do PDF
    for page in range(len(pdf_reader.pages)):
        # Obtém o objeto da página atual
        page_obj = pdf_reader.pages[page]

        # Extrai o texto da página atual
        page_text = page_obj.extract_text()

        # Escreve o texto extraído em um arquivo TXT
        with open(file_path + '.txt', "a",  encoding="utf-8") as text_file:
            text_file.write(page_text)
            txt_pdf += page_text + ' '

    # Fecha o arquivo PDF
    pdf_file.close()

    file_path = os.path.join(dir_, str(ano), pdf+'.txt')
    #print(txt_pdf)
    txt_pdf  = re.sub(r'/g\d+/', '', txt_pdf)
    txt_pdf  = re.sub(r'g/', '', txt_pdf)
    txt_pdf  = re.sub(r'/g/', '', txt_pdf)
    txt_pdf = re.sub(r'\{2,}', "", txt_pdf)
    txt_pdf = re.sub(r'\/\d+', "", txt_pdf)
    txt_pdf = re.sub(r'#\d+', "", txt_pdf)
    txt_pdf = txt_pdf.replace('#2/', '')
    txt_pdf = txt_pdf.replace('/', '')
    #*SA0175AZ6*
    #*DO0525AM31*
    #*010175AZ22*
    txt_pdf = re.sub(r"\*2017[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r"\*[A-Z]{2}\d{4}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r"\*\d{6}[A-Z]{2}\d+\*", '', txt_pdf)
    txt_pdf = re.sub(r'\*((SA)(CINZ)|(ROSA)|(BRAN)|(AMAR)|(AZUL)|(Amar)|(amar)|(cinz)|(azul)|(rosa)|(bran))[0-9]+((AZ)(dom)|(sab)|(DOM)|(SAB))\d+\*', '', txt_pdf)
    txt_pdf = txt_pdf.replace('2ª APLICAÇÃO', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 1 a 45', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 46 a 90', '')
    txt_pdf = txt_pdf.replace('LINGUAGENS E CÓDIGOS E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 91 a 135', '')
    txt_pdf = txt_pdf.replace('MATEMÁTICA E SUAS TECNOLOGIAS', '')
    txt_pdf = txt_pdf.replace('Questões de 136 a 180', '')
    txt_pdf = txt_pdf.replace('RASCUNHO DA REDAÇÃO', '')
    txt_pdf = txt_pdf.replace('Transcreva a sua Redação para a Folha de Redação.', '')
    txt_pdf = txt_pdf.replace('RASCUNHO\nDA REDAÇÃO1', '')
    txt_pdf = txt_pdf.replace('234567891011\n12\n131415161718192021222324252627282930', '')
    txt_pdf  = re.sub(r"(LC)|(MT)|(CN)|(CH) - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r" - (1º)|(2º) dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf  = re.sub(r"dia \| Caderno \d+ - ((AMARELO)|(AZUL)|(CINZA)|(ROSA)|(BRANCO)) - Página \d+", '', txt_pdf)
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZ', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA NATUREZAE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('AE SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('E SUAS TEOLOGIAS', '')
    txt_pdf = txt_pdf.replace('CIÊNCIAS DA N ATUREZA', '')
    txt_pdf = txt_pdf.replace('2014', '')
    txt_pdf = txt_pdf.replace('\n2015', '')
    txt_pdf = txt_pdf.replace('. 2015', '')
    txt_pdf = txt_pdf.replace('.  2015', '')
    txt_pdf = txt_pdf.replace('- 2017', '')
    txt_pdf = txt_pdf.replace('23456789101112131415161718192021222324252627282930', '')
    txt_pdf = txt_pdf.replace('RASCUNH\n DA REDAÇÃO 1', '')
    txt_pdf = txt_pdf.replace(' - Linguagens, Códigos e suas Tecnologias', '')
    txt_pdf = txt_pdf.replace(' - Ciências Humanas e suas Tecnologias', '')
    txt_pdf = txt_pdf.replace('CH - 1°', '')
    txt_pdf = txt_pdf.replace('- 1°', '')
    txt_pdf = txt_pdf.replace('- 2°', '')
    txt_pdf = txt_pdf.replace('2ª aplicação', '')
    txt_pdf =  txt_pdf.replace(' - Ciências Humanas e suas T ecnologias', '')
    txt_pdf =  txt_pdf.replace(' - Linguagens, Códigos e suas T ecnologias', '')
    txt_pdf =  txt_pdf.replace(' - Ciências da Natureza e suas T ecnologias', '')
    txt_pdf =  txt_pdf.replace(' - Matemática e suas T ecnologias', '')
    #letras = ['\nA ', '\nB ', '\nC ', '\nD ', '\nE ']
    #letras = ['A\n ', 'B\n ', 'C\n ', 'D\n ', 'E\n']
    #re.findall(r'Questão 15(.*?)\nA.', texto)
    #letras = ['\nA.', '\nB.', '\nC.', '\nD.', '\nE.']
    letras = ['\nA', '\nB', '\nC', '\nD', '\nE']
    globals()['alternativa0'] = ''
    globals()['alternativa1'] = ''
    globals()['alternativa2'] = ''
    globals()['alternativa3'] = ''
    globals()['alternativa4'] = ''
    for i in range(num_primeira_questao, num_ultima_questao+1):
        print(i)
        if i < num_ultima_questao:
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO "+(str(0)+str(i+1) if i+1 < 10 else str(i+1)), re.DOTALL)
            #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)da Associação Brasileira para", re.DOTALL)
            #questao = pattern.findall(txt_pdf)
            #if len(questao) == 0:
                #pattern = re.compile(r"QUESTÃO "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)QUESTÃO \d+", re.DOTALL)
                #questao = pattern.findall(txt_pdf)
            #pattern = re.compile(r"Questão "+(str(0)+str(i) if i < 10 else str(i))+"(.*?)Questão \d+", re.DOTALL)
            pattern = re.compile(r"Questão "+str(i)+"(.*?)Questão \d+", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            if (len(questao) > 0):
                questao = questao[0]
            else:
                questao = ''
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            print('texto_base')
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                print(texto_base)
            else:
                texto_base = ''
            for j in range(len(letras)):
                if j < 4:
                    
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    #(?<=(A\n|\nA)).*?(?=B\n|\nB)
                    #pattern = re.compile(r'(?<=(\n'+letras[j]+' |'+letras[j]+'\n|\n'+letras[j]+')).*?(?=\n'+letras[j+1]+' |'+letras[j+1]+'\n|\n'+letras[j+1]+' )', re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = questao.replace('\n', '')
            print('questão')
            print(questao)
        else:
            pattern = re.compile(r"Questão "+str(i)+"(.*)", re.DOTALL)
            questao = pattern.findall(txt_pdf)
            questao = questao[0].strip()
            pattern = re.compile(r"(.*)"+letras[0], re.DOTALL)
            texto_base = pattern.findall(questao)
            if (len(texto_base) > 0):
                texto_base = re.sub(r'\s+', ' ', texto_base[0], flags=re.DOTALL)
                
                print(texto_base)
            else:
                texto_base = ''
                
            print('alternativas')
            for j in range(len(letras)):
                if j < 4:
                    
                    pattern = re.compile(r''+letras[j]+"(.*?)"+letras[j+1], re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
                else:
                    pattern = re.compile(r''+letras[j]+"(.*)", re.DOTALL)
                    alternativa = pattern.findall(questao)
                    if (len(alternativa) > 0):
                        alternativa = re.sub(r'\s+', ' ', alternativa[0], flags=re.DOTALL)
                        alternativa = alternativa.replace('23456789101112131415161718192021222324252627282930', '')
                        alternativa = alternativa.replace('RASCUNHO DA REDAÇÃO 1', '')
                        alternativa = alternativa.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
                        alternativa = alternativa.replace('0101 4 9 10321 1°', '')
                        alternativa = alternativa.replace('0102 1° 9 10321', '')
                        alternativa =  alternativa.replace('1° 9 10321 0104', '')
                        alternativa =  alternativa.replace('2° 0201 56789101112', '')
                        alternativa =  alternativa.replace('2° 0203 56789101112', '')
                        alternativa =  alternativa.replace('2° 0202 56789101112', '')
                        alternativa =  alternativa.replace('0204 56789101112 2°', '')
                        alternativa =  alternativa.replace('2ª aplicação 2ª aplicação', '')
                        alternativa =  alternativa.replace('1° 0103  4 9 10321', '')
                        print(j, alternativa)
                        globals()['alternativa'+str(j)] = alternativa
                    else:
                        globals()['alternativa'+str(j)] = ''
            questao = re.sub(r'\s+', ' ', questao, flags=re.DOTALL)
            questao = questao.replace('RAScUnHODA REDAÇÃO2015123456789101112131415161718192021222324252627282930', '')
            questao = questao.replace('RAScUnHO DA REDAÇÃO2015 1 2 3 4 56 7 8910111213141516 17 18192021222324252627 28 29 30', '')
            questao = questao.replace('RASCUNHO DA REDAÇÃO 1', '')
            questao = questao.replace('2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30', '')
            questao = questao.replace('0101 4 9 10321 1°', '')
            questao =  questao.replace('0102 1° 9 10321', '')
            questao =  questao.replace('1° 9 10321 0104', '')
            questao =  questao.replace('2° 0201 56789101112', '')
            questao =  questao.replace('2° 0203 56789101112', '')
            questao =  questao.replace('2° 0202 56789101112', '')
            questao =  questao.replace('0204 56789101112 2°', '')
            questao =  questao.replace('2ª aplicação 2ª aplicação', '')
            questao =  questao.replace('1° 0103  4 9 10321', '')
            questao =  questao.replace(' - Ciências Humanas e suas T ecnologias', '')
            questao =  questao.replace(' - Linguagens, Códigos e suas T ecnologias', '')
            print(questao)
        dados.append([ano, cor, meio_aplicacao, i, areas[0], cos_prova[0],  questao, texto_base, globals()['alternativa0'], globals()['alternativa1'], globals()['alternativa2'], globals()['alternativa3'], globals()['alternativa4']])
    
    #print(dados)
    
    df_dados = pd.DataFrame(dados)
    df_dados = df_dados.rename(columns={0: 'ano', 1: 'cor', 2: 'meio_aplicacao', 3: 'num_questao', 4: 'area', 5: 'co_prova', 6: 'texto_questao', 7: 'texto_base', 8: 'alternativa_A', 9: 'alternativa_B', 10: 'alternativa_C', 11: 'alternativa_D', 12: 'alternativa_E'})
    df_dados = df_dados.sort_values(by='num_questao', ascending = True).reset_index(drop = True)
    if (areas[0] == 'LC'):
        df_dados.loc[df_dados['num_questao'] <= 45, 'area'] = areas[0]
        df_dados.loc[df_dados['num_questao'] <= 45, 'co_prova'] = cos_prova[0]
        df_dados.loc[df_dados['num_questao'] > 45, 'area'] = areas[1]
        df_dados.loc[df_dados['num_questao'] > 45, 'co_prova'] = cos_prova[1]
        print(np.sort(df_dados.num_questao.unique()))
        print(df_dados.area.unique())
        print(df_dados.co_prova.unique())
    elif (areas[0] == 'CN'):
        df_dados.loc[df_dados['num_questao'] <= 135, 'area'] = areas[0]
        df_dados.loc[df_dados['num_questao'] <= 135, 'co_prova'] = cos_prova[0]
        df_dados.loc[df_dados['num_questao'] > 135, 'area'] = areas[1]
        df_dados.loc[df_dados['num_questao'] > 135, 'co_prova'] = cos_prova[1]
        print(np.sort(df_dados.num_questao.unique()))
        print(df_dados.area.unique())
        print(df_dados.co_prova.unique())
    
    df_dados.to_csv(file_path + '.csv', sep = ';', index = False)

    return file_path, df_dados

In [ ]:
dados_21 = [['ENEM_2021_DIGITAL_CAD_01_DIA_1_AZUL_INGLES.pdf', 'AZUL', ['LC', 'CH'], [889, 879], [6, 90]],
        ['ENEM_2021_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES.pdf', 'AMARELA' , ['LC', 'CH'],[890, 880], [6, 90]],
        ['ENEM_2021_DIGITAL_CAD_04_DIA_1_ROSA_INGLES.pdf', 'ROSA' , ['LC', 'CH'],[891, 882], [6, 90]],
        ['ENEM_2021_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES.pdf', 'BRANCA' , ['LC', 'CH'],[892, 881], [6, 90]],
        ['ENEM_2021_DIGITAL_CAD_05_DIA_2_AMARELO.pdf', 'AMARELA' , ['CN', 'MT'],[910, 900], [91, 180]],
        ['ENEM_2021_DIGITAL_CAD_06_DIA_2_CINZA.pdf', 'CINZA' , ['CN', 'MT'],[911, 902], [91, 180]],
        ['ENEM_2021_DIGITAL_CAD_07_DIA_2_AZUL.pdf', 'AZUL' , ['CN', 'MT'],[909, 899], [91, 180]],
        ['ENEM_2021_DIGITAL_CAD_08_DIA_2_ROSA.pdf', 'ROSA' , ['CN', 'MT'],[912, 901], [91, 180]]]
dados_21

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_01_DIA_1_AZUL_INGLES = extrai_questoes_11('_itens', 2021, dados_21[0][0], dados_21[0][1], 1, [dados_21[0][2][0], dados_21[0][2][1]], [dados_21[0][3][0], dados_21[0][3][1]],  dados_21[0][4][0], dados_21[0][4][1])

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES = extrai_questoes_11('_itens', 2021, dados_21[1][0], dados_21[1][1], 1, [dados_21[1][2][0], dados_21[1][2][1]], [dados_21[1][3][0], dados_21[1][3][1]],  dados_21[1][4][0], dados_21[1][4][1])

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_04_DIA_1_ROSA_INGLES = extrai_questoes_11('_itens', 2021, dados_21[2][0], dados_21[2][1], 1, [dados_21[2][2][0], dados_21[2][2][1]], [dados_21[2][3][0], dados_21[2][3][1]],  dados_21[2][4][0], dados_21[2][4][1])

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES = extrai_questoes_11('_itens', 2021, dados_21[3][0], dados_21[3][1], 1, [dados_21[3][2][0], dados_21[3][2][1]], [dados_21[3][3][0], dados_21[3][3][1]],  dados_21[3][4][0], dados_21[3][4][1])

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_05_DIA_2_AMARELO = extrai_questoes_11('_itens', 2021, dados_21[4][0], dados_21[4][1], 1, [dados_21[4][2][0], dados_21[4][2][1]], [dados_21[4][3][0], dados_21[4][3][1]],  dados_21[4][4][0], dados_21[4][4][1])

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_06_DIA_2_CINZA = extrai_questoes_11('_itens', 2021, dados_21[5][0], dados_21[5][1], 1, [dados_21[5][2][0], dados_21[5][2][1]], [dados_21[5][3][0], dados_21[5][3][1]],  dados_21[5][4][0], dados_21[5][4][1])

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_07_DIA_2_AZUL = extrai_questoes_11('_itens', 2021, dados_21[6][0], dados_21[6][1], 1, [dados_21[6][2][0], dados_21[6][2][1]], [dados_21[6][3][0], dados_21[6][3][1]],  dados_21[6][4][0], dados_21[6][4][1])

In [ ]:
file_path, df_ENEM_2021_DIGITAL_CAD_08_DIA_2_ROSA = extrai_questoes_11('_itens', 2021, dados_21[7][0], dados_21[7][1], 1, [dados_21[7][2][0], dados_21[7][2][1]], [dados_21[7][3][0], dados_21[7][3][1]],  dados_21[7][4][0], dados_21[7][4][1])

In [ ]:
# Concatenação dos dataframes de 2021
df_prova_2021 = pd.concat([df_ENEM_2021_DIGITAL_CAD_01_DIA_1_AZUL_INGLES,
                          df_ENEM_2021_DIGITAL_CAD_02_DIA_1_AMARELO_INGLES,
                          df_ENEM_2021_DIGITAL_CAD_03_DIA_1_BRANCO_INGLES,
                          df_ENEM_2021_DIGITAL_CAD_04_DIA_1_ROSA_INGLES,
                          df_ENEM_2021_DIGITAL_CAD_05_DIA_2_AMARELO,
                          df_ENEM_2021_DIGITAL_CAD_06_DIA_2_CINZA,
                          df_ENEM_2021_DIGITAL_CAD_07_DIA_2_AZUL,
                          df_ENEM_2021_DIGITAL_CAD_08_DIA_2_ROSA], ignore_index=True, axis = 0)

# Resultado
df_prova_2021.shape

In [ ]:
np.sort(df_prova_2021.co_prova.unique())

In [ ]:
file = os.path.join('_itens', str(2021), 'ITENS_PROVA_2021')
df_itens_2021 = pd.read_csv(file + '.csv', sep = ';', encoding = 'ISO-8859-1')
df_itens_2021['id'] = df_itens_2021['CO_POSICAO'].apply(str) + '_' + df_itens_2021['SG_AREA'].apply(str) + '_' + df_itens_2021['TX_COR'].apply(str) 
df_prova_2021['id'] = df_prova_2021['num_questao'].apply(str) + '_' + df_prova_2021['area'].apply(str) + '_' + df_prova_2021['cor'].apply(str)
df_2021_merge = pd.merge(df_itens_2021, df_prova_2021, on='id', how='left')
df_2021_merge = df_2021_merge.drop(df_2021_merge.columns[20], axis = 1)
for col in df_2021_merge.columns:
    df_2021_merge = df_2021_merge.rename(columns = {col:col.lower()})
file = os.path.join('_itens', str(2021), '2021_ENEM_COMPLETA')
df_2021_merge.to_csv(file + '.csv', sep = ';', index = False)

### reunir em um dataframe todos os dados coletados

In [ ]:
# especificação dos tipos de cada coluna
'''dtype = {'nu_param_a': float, 'nu_param_b': float, 'nu_param_c': float}

df = pd.read_csv('arquivo.csv', dtype=dtype)'''

# Concatenação dos dataframes
df_itens_geral = pd.concat([pd.read_csv(os.path.join('_itens', '2010', '2010_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2011', '2011_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2012','2012_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2013','2013_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2014','2014_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2015','2015_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2017','2017_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2019','2019_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2020','2020_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8'),
                            pd.read_csv(os.path.join('_itens', '2021','2021_ENEM_COMPLETA') + '.csv', sep = ';', encoding = 'UTF-8')], ignore_index=True, axis = 0)

# Resultado
df_itens_geral.shape

In [ ]:
df_itens_geral.columns

In [ ]:
df_itens_geral = df_itens_geral.drop('co_prova.1', axis = 1).reset_index(drop = True)

In [ ]:
indexs = df_itens_geral[df_itens_geral['nu_param_a'].str.contains(',', na=False)].nu_param_a.index
for i in indexs:
    df_itens_geral.loc[i, 'nu_param_a'] = df_itens_geral.loc[i, 'nu_param_a'].replace(',', '.')
    
indexs = df_itens_geral[df_itens_geral['nu_param_b'].str.contains(',', na=False)].nu_param_b.index
for i in indexs:
    df_itens_geral.loc[i, 'nu_param_b'] = df_itens_geral.loc[i, 'nu_param_b'].replace(',', '.')
    
indexs = df_itens_geral[df_itens_geral['nu_param_c'].str.contains(',', na=False)].nu_param_c.index
for i in indexs:
    df_itens_geral.loc[i, 'nu_param_c'] = df_itens_geral.loc[i, 'nu_param_c'].replace(',', '.')

In [ ]:
df_itens_geral.nu_param_a = df_itens_geral.nu_param_a.astype(float)
df_itens_geral.nu_param_b = df_itens_geral.nu_param_b.astype(float)
df_itens_geral.nu_param_c = df_itens_geral.nu_param_c.astype(float)

In [ ]:
df_itens_geral.isnull().sum()

In [ ]:
df_itens_geral.head()

In [ ]:
file = os.path.join('_itens', 'df_itens_geral')
df_itens_geral.to_csv(file + '.csv', sep = ';', index = False)

In [ ]:
df_itens_geral.co_item

In [ ]:
df_itens_geral1 = df_itens_geral.drop_duplicates(subset=['co_item'])

In [ ]:
df_itens_geral1 = df_itens_geral1.drop_duplicates(subset=['texto_questao'])

In [ ]:
df_itens_geral1 = df_itens_geral1[(~df_itens_geral1.nu_param_a.isnull()) & ((~df_itens_geral1.nu_param_b.isnull()) & (~df_itens_geral1.nu_param_c.isnull()) & (df_itens_geral1.tx_motivo_aban.isnull()) & (~df_itens_geral1.texto_questao.isnull()))]

In [ ]:
df_itens_geral1.isnull().sum()

In [ ]:
df_itens_geral1.shape

In [ ]:
file = os.path.join('_itens', 'df_itens_geral_validos')
df_itens_geral1.to_csv(file + '.csv', sep = ';', index = False)